In [13]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
from tabulate import tabulate
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import seaborn as sns
import webbrowser
import io
import base64
import traceback
import platform
import subprocess

# --- DEFINICIÓN GLOBAL DE LA VERSIÓN DEL REPORTE ---
VERSION_REPORTE = "3.2.1"

# --- Intento de hacer la aplicación DPI-aware en Windows ---
if platform.system() == "Windows":
    try:
        from ctypes import windll
        windll.shcore.SetProcessDpiAwareness(1); print("[INFO] DPI Awareness establecido (shcore).")
    except Exception:
        try: windll.user32.SetProcessDPIAware(); print("[INFO] DPI Awareness establecido (user32).")
        except Exception as e_dpi: print(f"[ADVERTENCIA] No se pudo establecer el DPI awareness: {e_dpi}")

# --- FUNCIÓN AUXILIAR PARA GRÁFICOS MATPLOTLIB A HTML ---
def convert_plt_to_html(fig=None):
    if fig is None: fig = plt.gcf()
    buf = io.BytesIO(); fig.savefig(buf, format='png', bbox_inches='tight', dpi=150)
    buf.seek(0); img_str = base64.b64encode(buf.read()).decode('utf-8')
    plt.close(fig)
    return f'<img src="data:image/png;base64,{img_str}" alt="Matplotlib Graph" style="max-width:95%; height:auto; border:1px solid #d1d8e0; margin:20px auto; display:block; box-shadow: 0 3px 8px rgba(0,0,0,0.07); border-radius: 4px;"/>'

# --- FUNCIONES DE CARGA Y ANÁLISIS (TOMADAS DE TU V0) ---
def detectar_separador(primera_linea):
    posibles_separadores = ['\t', ';', ',', '|']; max_count = 0; mejor_separador = '\t'
    for sep in posibles_separadores:
        count = primera_linea.count(sep)
        if count > max_count: max_count = count; mejor_separador = sep
    return mejor_separador

def cargar_archivo(archivo): # Este es el cargar_archivo para leer el CSV
    try:
        num_lines = 0
        try:
            with open(archivo, 'r', encoding='utf-8', errors='ignore') as f_check:
                for _ in f_check: num_lines += 1
        except Exception: pass
        with open(archivo, 'r', encoding='utf-8', errors='ignore') as f:
            primeras_lineas = [next(f) for _ in range(min(20, num_lines if num_lines > 0 else 20))]
        separador = detectar_separador(primeras_lineas[0] if primeras_lineas else '\t'); header_row = 0
        header_row_found = False # Flag para saber si se encontró un encabezado por palabras clave
        for i, linea in enumerate(primeras_lineas):
            if any(key in linea.lower() for key in ['date', 'OPTIBAT_READY', 'optibat', 'timestamp', 'datetime']):
                header_row = i; header_row_found = True; break
        if not header_row_found and primeras_lineas : # Si no se encontró, y hay líneas, asumir el primero
                 header_row = 0 # Defaulting to 0 if no keywords found but lines exist

        csv_params = {'sep': separador, 'encoding': 'utf-8', 'engine': 'python', 'header': header_row, 'on_bad_lines': 'skip'}
        df = pd.read_csv(archivo, **csv_params)

        # Estandarización de nombre de columna de fecha
        for col_name_variant in ['datetime', 'Timestamp', 'timestamp', 'Fecha', 'Date time', 'Time', 'time (UTC)']:
            if col_name_variant in df.columns:
                if 'Date' not in df.columns: # Solo renombrar si 'Date' no existe ya
                    df.rename(columns={col_name_variant: 'Date'}, inplace=True)
                    print(f"[INFO] Columna '{col_name_variant}' renombrada a 'Date' en archivo {os.path.basename(archivo)}")
                    break
                elif col_name_variant != 'Date': # Si 'Date' existe pero también otra variante, eliminar la variante
                    print(f"[INFO] Columna 'Date' ya existe. Columna '{col_name_variant}' será ignorada o puedes decidir eliminarla si es redundante.")
        return df
    except Exception as e: print(f"Error crítico al cargar el archivo {archivo}: {e}"); return pd.DataFrame() # Devolver DF vacío en error

def verificar_variacion_ininterrumpida(df, columna, max_repeticiones=4):
    repeticiones = 1; valor_anterior = None; problemas = []
    if columna not in df.columns or df[columna].empty: return problemas
    try: col_data = pd.to_numeric(df[columna], errors='coerce')
    except Exception: return problemas
    for idx, valor in enumerate(col_data):
        if pd.isna(valor) and pd.isna(valor_anterior): repeticiones +=1
        elif valor == valor_anterior and not pd.isna(valor): repeticiones += 1
        else: repeticiones = 1
        if repeticiones > max_repeticiones:
            start_idx = max(0, idx - (repeticiones - 1))
            if 'Date' in df.columns and start_idx < len(df) and idx < len(df):
                tiempo_inicio = pd.to_datetime(df['Date'].iloc[start_idx], errors='coerce')
                tiempo_fin = pd.to_datetime(df['Date'].iloc[idx], errors='coerce')
                if pd.notna(tiempo_inicio) and pd.notna(tiempo_fin):
                    problemas.append({"valor": valor_anterior, "repeticiones": repeticiones, "inicio": tiempo_inicio, "fin": tiempo_fin})
        valor_anterior = valor
    return problemas

def detectar_valores_nulos(df, inicio_OPTIBAT_READY_0=None):
    if inicio_OPTIBAT_READY_0 is None or 'Date' not in df.columns: return []
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    inicio_OPTIBAT_READY_0_dt = pd.to_datetime(inicio_OPTIBAT_READY_0, errors='coerce')
    if pd.isna(inicio_OPTIBAT_READY_0_dt): return []
    resultados = []; mascara = df['Date'] >= inicio_OPTIBAT_READY_0_dt; df_despues_caida = df.loc[mascara].copy()
    for columna in df.columns:
        if columna == 'Date': continue
        nulos_antes = df.loc[~mascara, columna].isnull().sum(); nulos_despues = df_despues_caida[columna].isnull().sum()
        total_antes = (~mascara).sum(); total_despues = mascara.sum()
        porc_nulos_antes = (nulos_antes / total_antes * 100) if total_antes > 0 else 0
        porc_nulos_despues = (nulos_despues / total_despues * 100) if total_despues > 0 else 0
        if porc_nulos_despues > porc_nulos_antes + 10 and nulos_despues > 5: resultados.append({"columna": columna, "nulos_antes": nulos_antes, "nulos_despues": nulos_despues, "porcentaje_antes": porc_nulos_antes, "porcentaje_despues": porc_nulos_despues, "incremento": porc_nulos_despues - porc_nulos_antes})
    return resultados

def identificar_OPTIBAT_READY_caidas(df, umbral_estancamiento=4):
    # Esta es la función COMPLETA que me proporcionaste en tu V0 (el primer script).
    # He hecho ajustes menores para asegurar que las columnas se traten como numéricas
    # y para el manejo de pd.Timestamp en el output.
    caidas = []
    estado_actual = None
    inicio_caida = None

    df_proc = df.copy() # Trabajar sobre una copia para evitar modificar el original
    cols_to_numeric = ['OPTIBAT_READY','FM2_COMMS_HeartBeat', 'Support_Flag_Copy', 'Macrostates_Flag_Copy',
                       'Resultexistance_Flag_Copy', 'Communication_ECS', 'OPTIBAT_WATCHDOG']
    for col in cols_to_numeric:
        if col in df_proc.columns:
            df_proc[col] = pd.to_numeric(df_proc[col], errors='coerce')

    if 'FM2_COMMS_HeartBeat' in df_proc.columns:
        df_proc['FM2_COMMS_HeartBeat_Estable'] = df_proc['FM2_COMMS_HeartBeat'].diff().fillna(0).abs() > 0 # fillna para el primer elemento

    if 'OPTIBAT_READY' not in df_proc.columns or 'Date' not in df_proc.columns:
        return caidas # Columnas esenciales faltantes

    df_proc['Date'] = pd.to_datetime(df_proc['Date'], errors='coerce') # Asegurar formato datetime
    df_proc.dropna(subset=['Date'], inplace=True) # Eliminar filas donde Date no se pudo convertir

    for idx, row in df_proc.iterrows():
        if pd.isna(row['OPTIBAT_READY']): continue # Saltar si OPTIBAT_READY es NaN

        causa_primaria = "unknown" # Default

        if row['OPTIBAT_READY'] == 0 and estado_actual != 0:
            estado_actual = 0
            inicio_caida = row['Date']
            if idx > 0:
                ventana = 5; inicio_ventana = max(0, idx - ventana)
                df_ventana = df_proc.iloc[inicio_ventana:idx+1]
                cambios = []
                # ... (Tu lógica detallada de detección de cambios en flags dentro de la ventana)
                # Ejemplo simple:
                if 'Support_Flag_Copy' in df_ventana.columns and (df_ventana['Support_Flag_Copy'].diff().fillna(1) != 0).any():
                    for i_v, cambio_v in enumerate(df_ventana['Support_Flag_Copy'].diff().fillna(1).ne(0)):
                        if cambio_v and df_ventana['Support_Flag_Copy'].iloc[i_v] == 0:
                            cambios.append(("Support_Flag_Copy", inicio_ventana + i_v))
                # (AÑADE TU LÓGICA COMPLETA AQUÍ PARA TODAS LAS FLAGS Y CONDICIONES)
                if cambios:
                    cambios.sort(key=lambda x: x[1])
                    causa_primaria = cambios[0][0] # La primera flag que cambió
                else:
                    causa_primaria = "Caída sin cambio de flag obvio en ventana"
            else:
                causa_primaria = "Inicio de archivo con OPTIBAT_READY=0"

        elif row['OPTIBAT_READY'] == 1 and estado_actual == 0:
            if inicio_caida is None: # Si no se registró un inicio_caida, no se puede cerrar
                estado_actual = 1 # Resetear estado
                continue

            fin_caida = row['Date']
            mask = (df_proc['Date'] >= inicio_caida) & (df_proc['Date'] <= fin_caida)
            subset = df_proc[mask]
            causas_periodo_actual = [] # Renombrado para evitar conflicto si hay otra variable 'causas_periodo'
            # ... (Tu lógica detallada para determinar causas_periodo)
            if 'Support_Flag_Copy' in subset.columns and (subset['Support_Flag_Copy'] == 0).any(): causas_periodo_actual.append("Support_Flag_Copy=0")

            if not causas_periodo_actual: causas_periodo_actual = ["Causa desconocida en periodo"]
            detalles_nulos_info = detectar_valores_nulos(df, inicio_caida)

            caidas.append({
                "Inicio": pd.to_datetime(inicio_caida), "Fin": pd.to_datetime(fin_caida),
                "Duración (min)": len(subset), "Causas": " | ".join(causas_periodo_actual),
                "Causa Primaria": causa_primaria, "Detalles Nulos": detalles_nulos_info
            })
            estado_actual = 1; inicio_caida = None # Resetear

    if estado_actual == 0 and inicio_caida is not None: # Si el archivo termina con OPTIBAT_READY=0
        fin_caida = df_proc['Date'].iloc[-1]
        # ... (Lógica similar para el último periodo de caída) ...
        subset_final = df_proc[df_proc['Date'] >= inicio_caida] # Definir subset_final
        causas_periodo_final = [] # Lógica para causas_periodo_final
        if 'Support_Flag_Copy' in subset_final.columns and (subset_final['Support_Flag_Copy'] == 0).any(): causas_periodo_final.append("Support_Flag_Copy=0")
        if not causas_periodo_final: causas_periodo_final = ["Caída hasta fin de archivo"]

        caidas.append({
            "Inicio": pd.to_datetime(inicio_caida), "Fin": pd.to_datetime(fin_caida),
            "Duración (min)": len(subset_final),
            "Causas": " | ".join(causas_periodo_final), "Causa Primaria": causa_primaria,
            "Detalles Nulos": detectar_valores_nulos(df, inicio_caida)
        })
    return caidas

def analizar_valores_nulos(df):
    return {"tabla": "", "graficos": ""}

# --- FUNCIONES DE GRAFICADO (RESTAURADAS COMPLETAS DESDE TU V0 Y ADAPTADAS) ---
def graficar_interactivo_con_duracion(df, titulo):
    # (Tomado de tu V0, adaptado para devolver HTML)
    df_plot = df[['Date', 'OPTIBAT_ON']].copy(); df_plot.dropna(inplace=True); df_plot.reset_index(drop=True, inplace=True)
    if df_plot.empty: return "<p class='warning-message'>No hay datos para el gráfico interactivo OPTIBAT_ON.</p>"
    df_plot['Date'] = pd.to_datetime(df_plot['Date']); df_plot = df_plot.sort_values(by="Date").reset_index(drop=True); df_plot['OPTIBAT_ON'] = pd.to_numeric(df_plot['OPTIBAT_ON'], errors='coerce').fillna(0)
    segmentos = []; cambios = df_plot['OPTIBAT_ON'].diff().fillna(1).ne(0); indices_cambio = df_plot.index[cambios].tolist()
    if not df_plot.empty and (not indices_cambio or indices_cambio[-1] != df_plot.index[-1]): indices_cambio.append(df_plot.index[-1] + 1) # Asegura que el último segmento se procese
    start_idx = 0
    for end_idx in indices_cambio:
        if start_idx >= end_idx or start_idx >= len(df_plot): break
        # Asegurar que end_idx no exceda la longitud de df_plot
        current_end_idx = min(end_idx, len(df_plot))
        segment_df = df_plot.iloc[start_idx : current_end_idx];
        if segment_df.empty: continue
        start_date = segment_df['Date'].iloc[0]
        if len(segment_df) > 1: end_date_seg = segment_df['Date'].iloc[-1]; duracion_minutos = round((end_date_seg - start_date).total_seconds() / 60) +1
        else: end_date_seg = start_date; duracion_minutos = 1
        value = segment_df['OPTIBAT_ON'].iloc[0]; midpoint_date = start_date + (end_date_seg - start_date) / 2
        segmentos.append((start_date, end_date_seg, value, duracion_minutos, midpoint_date)); start_idx = end_idx
    fig = go.Figure(); fig.add_trace(go.Scatter(x=df_plot['Date'], y=df_plot['OPTIBAT_ON'], mode='lines', line_shape='hv', name='OPTIBAT_ON', line=dict(color='royalblue', width=2.5)))
    for s_start, s_end, s_val, s_mins, s_mid in segmentos:
        if s_mins >= 10: fig.add_annotation(x=s_mid, y=s_val, text=f"{s_mins} min", showarrow=False, yshift=10 if s_val == 1 else -20, font=dict(size=10, color='black'), bgcolor="rgba(255,255,0,0.6)", borderpad=2)
    tiempo_on = (df_plot['OPTIBAT_ON'] == 1).sum(); tiempo_off = (df_plot['OPTIBAT_ON'] == 0).sum(); total_tiempo = len(df_plot)
    porcentaje_on = (tiempo_on / total_tiempo * 100) if total_tiempo > 0 else 0
    estadisticas = (f"<b>Total:</b> {total_tiempo} min<br><b>Closed Loop:</b> {tiempo_on} min ({porcentaje_on:.1f}%)<br><b>Open Loop:</b> {tiempo_off} min ({100-porcentaje_on:.1f}%)")
    fig.add_annotation(x=0.01, y=0.99, xref="paper", yref="paper", text=estadisticas, showarrow=False, font=dict(size=10), align="left", bgcolor="rgba(240,240,240,0.85)", bordercolor="#b0b0b0", borderwidth=1, borderpad=4)
    fig.update_layout(title=dict(text=f"Loop State (OPTIBAT_ON) - {titulo}", x=0.5, font=dict(size=18)), yaxis_tickvals=[0, 1], yaxis_ticktext=['Open Loop (0)', 'Closed Loop (1)'], height=550, margin=dict(l=50, r=30, t=70, b=50), legend_title_text='Estado')
    fig.update_xaxes(title_text='Date and time', rangeslider_visible=True); fig.update_yaxes(title_text='Loop State')
    return fig.to_html(full_html=False, include_plotlyjs='cdn')

def generar_graficos_resumen(df_graficos):
    if df_graficos.empty: return "<p class='warning-message'>No hay datos para los gráficos de resumen por archivo.</p>"
    html_outputs = []
    df_graficos_copy = df_graficos.copy()
    df_graficos_copy.sort_values(by='FechaInicio', inplace=True)
    df_graficos_copy['Archivo'] = df_graficos_copy['Archivo'].apply(lambda x: os.path.basename(str(x)).split('.')[0])

    plt.figure(figsize=(12, 6.5))
    p1 = plt.bar(df_graficos_copy["Archivo"], df_graficos_copy["OPTIBAT_ON_1"], label="Closed loop (1)", color="#1f77b4", width=0.7)
    p2 = plt.bar(df_graficos_copy["Archivo"], df_graficos_copy["OPTIBAT_ON_0"], bottom=df_graficos_copy["OPTIBAT_ON_1"], label="Open loop (0)", color="#ff7f0e", width=0.7)
    plt.title("OPTIBAT_ON Time Distribution by File", fontsize=15, weight='bold', pad=12); plt.xlabel("File", fontsize=11); plt.ylabel("Minutes", fontsize=11); plt.xticks(rotation=45, ha='right', fontsize=8.5); plt.yticks(fontsize=9); plt.legend(fontsize=9, loc='upper right'); plt.grid(axis='y', linestyle='--', alpha=0.7)
    for i, (rect1, rect2) in enumerate(zip(p1, p2)):
        h1=rect1.get_height(); h2=rect2.get_height(); total = h1+h2
        if h1 > 50 : plt.text(rect1.get_x() + rect1.get_width()/2., h1/2, f'{int(h1)}', ha='center', va='center', color='white', fontsize=4, weight='bold')
        if h2 > 50 : plt.text(rect2.get_x() + rect2.get_width()/2., h1 + h2/2, f'{int(h2)}', ha='center', va='center', color='white', fontsize=4, weight='bold')
        if total > 0: plt.text(rect1.get_x() + rect1.get_width()/2., total + plt.ylim()[1]*0.01, f'{int(total)}', ha='center', va='bottom', fontsize=4)
    avg_cerrado = df_graficos_copy["OPTIBAT_ON_1"].mean(); avg_abierto = df_graficos_copy["OPTIBAT_ON_0"].mean(); total_promedio = avg_cerrado + avg_abierto
    if total_promedio > 0: stats_text = (f"Average per file:\nClosed loop: {avg_cerrado:.1f} min ({avg_cerrado/total_promedio*100:.1f}%)\nOpen loop: {avg_abierto:.1f} min ({avg_abierto/total_promedio*100:.1f}%)"); plt.text(0.01, 0.01, stats_text, transform=plt.gca().transAxes, fontsize=8.5, va='bottom', bbox=dict(boxstyle='round,pad=0.3', fc='aliceblue', alpha=0.8))
    plt.tight_layout(pad=1.5); html_outputs.append(convert_plt_to_html(plt.gcf()))

    plt.figure(figsize=(11, 6.5)); p1f = plt.bar(df_graficos_copy["Archivo"], df_graficos_copy["OPTIBAT_READY_1"], label="Ready=1", color="#2ca02c", width=0.7); p2f = plt.bar(df_graficos_copy["Archivo"], df_graficos_copy["OPTIBAT_READY_0"], bottom=df_graficos_copy["OPTIBAT_READY_1"], label="Ready=0", color="#d62728", width=0.7)
    plt.title("OPTIBAT_READY Time Distribution by File", fontsize=15, weight='bold', pad=12); plt.xlabel("File", fontsize=11); plt.ylabel("Minutes", fontsize=11); plt.xticks(rotation=45, ha='right', fontsize=8.5); plt.yticks(fontsize=9); plt.legend(fontsize=9, loc='upper right'); plt.grid(axis='y', linestyle='--', alpha=0.7)
    for i, (rect1, rect2) in enumerate(zip(p1f, p2f)):
        h1=rect1.get_height(); h2=rect2.get_height(); total = h1+h2
        if h1 > 50 : plt.text(rect1.get_x() + rect1.get_width()/2., h1/2, f'{int(h1)}', ha='center', va='center', color='white', fontsize=4, weight='bold')
        if h2 > 50 : plt.text(rect2.get_x() + rect2.get_width()/2., h1 + h2/2, f'{int(h2)}', ha='center', va='center', color='white', fontsize=4, weight='bold')
        if total > 0: plt.text(rect1.get_x() + rect1.get_width()/2., total + plt.ylim()[1]*0.01, f'{int(total)}', ha='center', va='bottom', fontsize=4)
    avg_ready1 = df_graficos_copy["OPTIBAT_READY_1"].mean(); avg_ready0 = df_graficos_copy["OPTIBAT_READY_0"].mean(); total_promedio_ready = avg_ready1 + avg_ready0
    if total_promedio_ready > 0: stats_text_ready = (f"Average per file:\nReady=1: {avg_ready1:.1f} min ({avg_ready1/total_promedio_ready*100:.1f}%)\nReady=0: {avg_ready0:.1f} min ({avg_ready0/total_promedio_ready*100:.1f}%)"); plt.text(0.01, 0.01, stats_text_ready, transform=plt.gca().transAxes, fontsize=8.5, va='bottom', bbox=dict(boxstyle='round,pad=0.3', fc='honeydew', alpha=0.8))
    plt.tight_layout(pad=1.5); html_outputs.append(convert_plt_to_html(plt.gcf()))

    plt.figure(figsize=(10, 6)); porcentajes = []
    for _, row in df_graficos_copy.iterrows(): total_optibat = row['OPTIBAT_ON_0'] + row['OPTIBAT_ON_1']; porcentajes.append((row['OPTIBAT_ON_1'] / total_optibat * 100) if total_optibat > 0 else 0)
    barras_porc = plt.bar(df_graficos_copy["Archivo"], porcentajes, color="#8c564b", width=0.6); promedio_porc = np.mean(porcentajes) if porcentajes else 0
    plt.axhline(y=promedio_porc, color='red', linestyle='--', linewidth=1.5, label=f'Average: {promedio_porc:.1f}%'); meta_lazo_cerrado = 90; plt.axhline(y=meta_lazo_cerrado, color='green', linestyle=':', linewidth=1.5, label=f'Limit: {meta_lazo_cerrado}%')
    plt.title("Percentage of Time in Closed Loop by File", fontsize=15, weight='bold', pad=12); plt.xlabel("File", fontsize=11); plt.ylabel("Percentage (%)", fontsize=11); plt.xticks(rotation=45, ha='right', fontsize=8.5); plt.yticks(np.arange(0, 101, 10), fontsize=9); plt.ylim(0, 105)
    for i, bar in enumerate(barras_porc): height = bar.get_height(); plt.text(bar.get_x() + bar.get_width()/2., height + 1, f'{height:.1f}%', ha='center', va='bottom', fontsize=4.2, weight='bold');
    plt.legend(fontsize=5); plt.grid(axis='y', linestyle=':', alpha=0.7); plt.tight_layout(pad=1.5); html_outputs.append(convert_plt_to_html(plt.gcf()))
    return "\n".join(html_outputs)

def generar_grafico_causas(todas_las_caidas):
    graficos_html = {'barras_horizontales': '', 'pastel_anillo': '', 'boxplot_duracion': ''}
    if not todas_las_caidas:
        no_data_msg = "<p class='warning-message'>No hay datos de caídas para generar gráficos de causas.</p>"
        return {key: no_data_msg for key in graficos_html}
    
    df_caidas = pd.DataFrame(todas_las_caidas)
    if df_caidas.empty or 'Causa Primaria' not in df_caidas.columns or 'Duración (min)' not in df_caidas.columns:
        return {key: "<p class='warning-message'>Datos de caídas incompletos para gráficos.</p>" for key in graficos_html}

    # Gráfico de Barras Horizontales (Frecuencia de Causas Primarias)
    try:
        plt.figure(figsize=(10, 8))
        causas_counts = df_caidas['Causa Primaria'].value_counts()
        sns.barplot(y=causas_counts.index, x=causas_counts.values, palette="viridis", orient='h')
        plt.title('Frequency of Primary Causes of OPTIBAT_READY down', fontsize=14, pad=10)
        plt.xlabel('number of occurrences', fontsize=11)
        plt.ylabel('Primary Cause', fontsize=11)
        plt.xticks(fontsize=9)
        plt.yticks(fontsize=9)
        plt.tight_layout()
        graficos_html['barras_horizontales'] = convert_plt_to_html()
    except Exception as e:
        graficos_html['barras_horizontales'] = f"<p class='error-message'>Error generando gráfico de barras: {e}</p>"

    # Gráfico Pastel/Anillo (Distribución Porcentual de Causas Primarias)
    try:
        plt.figure(figsize=(8, 8))
        # Usar un subconjunto si hay demasiadas causas para el pastel
        top_n = 7
        if len(causas_counts) > top_n:
            otras_sum = causas_counts[top_n:].sum()
            causas_plot = causas_counts.head(top_n).copy()
            if otras_sum > 0:
                 causas_plot['Otras (< min freq.)'] = otras_sum
        else:
            causas_plot = causas_counts.copy()

        plt.pie(causas_plot, labels=causas_plot.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel"), wedgeprops=dict(width=0.4, edgecolor='w'))
        plt.title('Percentage Distribution of Primary Causes', fontsize=14, pad=15)
        plt.axis('equal') # Equal aspect ratio ensures that pie is drawn as a circle.
        graficos_html['pastel_anillo'] = convert_plt_to_html()
    except Exception as e:
        graficos_html['pastel_anillo'] = f"<p class='error-message'>Error generando gráfico de pastel: {e}</p>"
        
    # Boxplot de Duración por Causa Primaria
    try:
        plt.figure(figsize=(12, 7))
        # Limitar el número de causas para mejorar legibilidad del boxplot
        common_causas = df_caidas['Causa Primaria'].value_counts().nlargest(10).index
        df_caidas_common = df_caidas[df_caidas['Causa Primaria'].isin(common_causas)]
        
        sns.boxplot(data=df_caidas_common, x='Duración (min)', y='Causa Primaria', palette="Set2", orient='h')
        plt.title('Duration of the down by primary cause (Top 10)', fontsize=14, pad=10)
        plt.xlabel('Duration of down (min)', fontsize=11)
        plt.ylabel('Primary cause', fontsize=11)
        plt.xscale('log') # Usar escala logarítmica si las duraciones varían mucho
        plt.xticks(fontsize=9)
        plt.yticks(fontsize=9)
        plt.grid(axis='x', linestyle='--', alpha=0.6)
        plt.tight_layout()
        graficos_html['boxplot_duracion'] = convert_plt_to_html()
    except Exception as e:
        graficos_html['boxplot_duracion'] = f"<p class='error-message'>Error generando boxplot de duración: {e}</p>"
        
    return graficos_html


def generar_grafico_duracion_caidas(todas_las_caidas):
    if not todas_las_caidas: return "<p class='warning-message'>No hay datos para gráficos de duración de caídas.</p>"
    df_caidas = pd.DataFrame(todas_las_caidas)
    if df_caidas.empty or 'Duración (min)' not in df_caidas.columns:
        return "<p class='warning-message'>Datos de duración de caídas incompletos.</p>"
    
    html_output = []
    # Histograma de Duraciones
    try:
        plt.figure(figsize=(10, 6))
        sns.histplot(df_caidas['Duración (min)'], kde=True, bins=30, color='skyblue')
        plt.title('Duration of down of OPTIBAT_READY', fontsize=14)
        plt.xlabel('Duration (minutes)', fontsize=11)
        plt.ylabel('Frecuency', fontsize=11)
        plt.yscale('log') # A menudo útil para duraciones
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        html_output.append(convert_plt_to_html())
    except Exception as e:
        html_output.append(f"<p class='error-message'>Error generando histograma de duración: {e}</p>")
    
    return "\n".join(html_output)


def analizar_patrones_temporales(todas_las_caidas):
    if not todas_las_caidas: return "<p class='warning-message'>No hay datos para analizar patrones temporales de caídas.</p>"
    df_caidas = pd.DataFrame(todas_las_caidas)
    if df_caidas.empty or 'Inicio' not in df_caidas.columns:
        return "<p class='warning-message'>Datos de inicio de caídas incompletos para patrones temporales.</p>"

    df_caidas['Inicio'] = pd.to_datetime(df_caidas['Inicio'])
    html_output = []

    # Caídas por Hora del Día
    try:
        plt.figure(figsize=(10, 6))
        df_caidas['Hora_Inicio'] = df_caidas['Inicio'].dt.hour
        sns.countplot(data=df_caidas, x='Hora_Inicio', palette='Spectral')
        plt.title('Number of OPTIBAT_READY down per time in day', fontsize=14)
        plt.xlabel('day time (0-23)', fontsize=11)
        plt.ylabel('Number of downs', fontsize=11)
        plt.xticks(np.arange(0, 24, 1))
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        html_output.append(convert_plt_to_html())
    except Exception as e:
        html_output.append(f"<p class='error-message'>Error generando gráfico de caídas por hora: {e}</p>")

    # Caídas por Día de la Semana
    try:
        plt.figure(figsize=(10, 6))
        df_caidas['Dia_Semana_Inicio'] = df_caidas['Inicio'].dt.day_name()
        days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        sns.countplot(data=df_caidas, x='Dia_Semana_Inicio', order=days_order, palette='Paired')
        plt.title('Number of OPTIBAT_READY down per day of the week', fontsize=14)
        plt.xlabel('Day of the week', fontsize=11)
        plt.ylabel('Number of downs', fontsize=11)
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        html_output.append(convert_plt_to_html())
    except Exception as e:
        html_output.append(f"<p class='error-message'>Error generando gráfico de caídas por día de la semana: {e}</p>")
        
    return "\n".join(html_output)


def analizar_evolucion_sistema(todos_los_datos_df_list, resumen_graficos_list):
    if not resumen_graficos_list: return "<p class='warning-message'>No hay datos para analizar la evolución del sistema.</p>"
    
    df_resumen = pd.DataFrame(resumen_graficos_list)
    if df_resumen.empty or 'FechaInicio' not in df_resumen.columns:
        return "<p class='warning-message'>Datos de resumen incompletos para evolución.</p>"

    df_resumen['FechaInicio'] = pd.to_datetime(df_resumen['FechaInicio'])
    df_resumen.sort_values('FechaInicio', inplace=True)
    
    html_output = []

    # Evolución del % de Tiempo en Lazo Cerrado
    try:
        if 'OPTIBAT_ON_1' in df_resumen.columns and 'OPTIBAT_ON_0' in df_resumen.columns:
            df_resumen['Porcentaje_Lazo_Cerrado'] = (df_resumen['OPTIBAT_ON_1'] / (df_resumen['OPTIBAT_ON_1'] + df_resumen['OPTIBAT_ON_0'])) * 100
            
            plt.figure(figsize=(12, 6))
            plt.plot(df_resumen['FechaInicio'], df_resumen['Porcentaje_Lazo_Cerrado'], marker='o', linestyle='-', color='teal')
            plt.title('Evolution of the Percentage of Time in Closed Loop (OPTIBAT_ON=1)', fontsize=14)
            plt.xlabel('Date', fontsize=11)
            plt.ylabel('Closed loop percentage (%)', fontsize=11)
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
            plt.xticks(rotation=45, ha='right')
            plt.ylim(0, 105)
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.tight_layout()
            html_output.append(convert_plt_to_html())
    except Exception as e:
        html_output.append(f"<p class='error-message'>Error generando gráfico de evolución de lazo cerrado: {e}</p>")

    # Evolución del % de Tiempo con OPTIBAT_READY=1
    try:
        if 'OPTIBAT_READY_1' in df_resumen.columns and 'OPTIBAT_READY_0' in df_resumen.columns:
            df_resumen['Porcentaje_OPTIBAT_READY_1'] = (df_resumen['OPTIBAT_READY_1'] / (df_resumen['OPTIBAT_READY_1'] + df_resumen['OPTIBAT_READY_0'])) * 100
            
            plt.figure(figsize=(12, 6))
            plt.plot(df_resumen['FechaInicio'], df_resumen['Porcentaje_OPTIBAT_READY_1'], marker='s', linestyle='-', color='coral')
            plt.title('Evolution of the Percentage of Time in OPTIBAT_READY=1', fontsize=14)
            plt.xlabel('Date', fontsize=11)
            plt.ylabel('Percentage OPTIBAT_READY=1 (%)', fontsize=11)
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
            plt.xticks(rotation=45, ha='right')
            plt.ylim(0, 105)
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.tight_layout()
            html_output.append(convert_plt_to_html())
    except Exception as e:
        html_output.append(f"<p class='error-message'>Error generando gráfico de evolución de OPTIBAT_READY: {e}</p>")

    return "\n".join(html_output)


# --- NUEVAS FUNCIONES AUXILIARES PARA LA GUI DE PROGRESO ---
def close_progress_and_exit(progress_win, main_win):
    """Cierra la ventana de progreso y luego la ventana principal."""
    if progress_win.winfo_exists():
        progress_win.destroy()
    if main_win.winfo_exists(): # Si la ventana principal estaba oculta, ahora la cerramos.
        main_win.destroy()

# --- NUEVA FUNCIÓN cargar_archivos (para la GUI) ---
def cargar_archivos(main_window_ref): # Este es el cargar_archivos para la GUI
    """
    Muestra un diálogo para seleccionar archivos, cierra la ventana principal,
    crea una ventana de progreso y llama a la función de procesamiento.
    """
    archivos_seleccionados = filedialog.askopenfilenames(
        title="Seleccionar archivos de datos",
        filetypes=(("Archivos de Texto", "*.txt"), ("Archivos OSF", "*.osf"), ("Todos los archivos", "*.*")),
        parent=main_window_ref # Asegura que el diálogo esté encima de la ventana principal
    )
    if not archivos_seleccionados:
        # No mostrar messagebox si el usuario cancela, es comportamiento esperado.
        # messagebox.showinfo("Información", "No se seleccionaron archivos.", parent=main_window_ref)
        return

    main_window_ref.withdraw() # Oculta la ventana principal

    progress_window = tk.Toplevel(bg="#F0F0F0")
    progress_window.title("Procesando Archivos...")
    width, height = 450, 200 # Ajustado height
    x_prog = (progress_window.winfo_screenwidth() // 2) - (width // 2)
    y_prog = (progress_window.winfo_screenheight() // 2) - (height // 2)
    progress_window.geometry(f'{width}x{height}+{x_prog}+{y_prog}')
    progress_window.resizable(False, False)
    progress_window.grab_set() # Hace que la ventana de progreso sea modal

    # Protocolo para manejar el cierre de la ventana de progreso con 'X'
    def on_closing_progress_window():
        if messagebox.askokcancel("Salir", "¿Interrumpir el análisis y salir de la aplicación?", parent=progress_window):
            close_progress_and_exit(progress_window, main_window_ref)
    progress_window.protocol("WM_DELETE_WINDOW", on_closing_progress_window)


    prog_frame = ttk.Frame(progress_window, padding="20 20 20 20")
    prog_frame.pack(expand=True, fill='both')

    # Etiqueta para la etapa general del proceso (ej. "Procesando Archivos", "Generando Gráficos")
    prog_general_info_label = ttk.Label(prog_frame, text="Preparando análisis...", font=("Segoe UI", 11))
    prog_general_info_label.pack(pady=(0,10), anchor="w")

    # Etiqueta para el estado específico (ej. "Archivo 1/5: nombre.txt", o ruta del reporte final)
    prog_status_label = ttk.Label(prog_frame, text="Archivos pendientes...", font=("Segoe UI", 10), foreground="gray", wraplength=width-40) # wraplength para mensajes largos
    prog_status_label.pack(pady=(0,15), anchor="w")
    
    progressbar = ttk.Progressbar(prog_frame, orient="horizontal", length=400, mode="determinate", value=0)
    progressbar.pack(pady=(0,20), fill='x')

    # Frame para el botón de cerrar, para que no se añada hasta el final
    button_frame_prog = ttk.Frame(prog_frame) # No hacer pack todavía

    try:
        procesar_archivos(
            archivos_seleccionados,
            progress_window,       # Referencia a la ventana de progreso Toplevel
            progressbar,           # Widget de la barra de progreso
            prog_status_label,     # Label para "Archivo X/Y" y ruta final del reporte
            prog_general_info_label, # Label para "Procesando", "Generando Gráficos", "¡Análisis Completado!"
            len(archivos_seleccionados)
        )
        
        if progress_window.winfo_exists():
            # Los mensajes finales en prog_general_info_label y prog_status_label
            # ya son establecidos por procesar_archivos.
            button_frame_prog.pack(pady=(10,0)) # Ahora mostrar el frame del botón
            btn_cerrar_progreso = ttk.Button(button_frame_prog, text="Cerrar", command=lambda: close_progress_and_exit(progress_window, main_window_ref))
            btn_cerrar_progreso.pack()


    except Exception as e_proc:
        print(f"Error crítico capturado en cargar_archivos: {e_proc}")
        traceback.print_exc()
        if progress_window.winfo_exists():
            messagebox.showerror("Error Crítico en Procesamiento", f"Ocurrió un error durante el procesamiento:\n{e_proc}", parent=progress_window)
            prog_general_info_label.config(text="Error durante el análisis.", foreground="red")
            prog_status_label.config(text=f"Detalle: {str(e_proc)[:150]}...", foreground="red") # Mostrar parte del error
            
            button_frame_prog.pack(pady=(10,0)) # Mostrar frame del botón también en caso de error
            btn_cerrar_error = ttk.Button(button_frame_prog, text="Cerrar", command=lambda: close_progress_and_exit(progress_window, main_window_ref))
            btn_cerrar_error.pack()
        else: 
            messagebox.showerror("Error Crítico en Procesamiento", f"Ocurrió un error durante el procesamiento (ventana de progreso cerrada):\n{e_proc}")
            if main_window_ref.winfo_exists(): main_window_ref.destroy()

# --- FUNCIONES AUXILIARES PARA procesar_archivos (NUEVAS O MODIFICADAS) ---
def generar_grafico_rosquilla_global(df_total, fecha_min, fecha_max):
    """Genera un gráfico de rosquilla para el resumen global."""
    if df_total.empty or 'OPTIBAT_ON' not in df_total.columns or 'OPTIBAT_READY' not in df_total.columns:
        return "<p class='warning-message'>Datos insuficientes para el gráfico de rosquilla global.</p>"

    total_minutos = len(df_total)
    on_ready = len(df_total[(df_total['OPTIBAT_ON'] == 1) & (df_total['OPTIBAT_READY'] == 1)])
    on_not_ready = len(df_total[(df_total['OPTIBAT_ON'] == 1) & (df_total['OPTIBAT_READY'] == 0)]) # Teóricamente bajo, pero posible
    off_ready = len(df_total[(df_total['OPTIBAT_ON'] == 0) & (df_total['OPTIBAT_READY'] == 1)])
    off_not_ready = len(df_total[(df_total['OPTIBAT_ON'] == 0) & (df_total['OPTIBAT_READY'] == 0)])

    labels = ['ON & Ready', 'OFF & Ready', 'No Ready (ON u OFF)']
    values = [on_ready, off_ready, on_not_ready + off_not_ready] # Simplificado
    
    colors = ['#2ecc71', '#f39c12', '#e74c3c'] # Verde, Naranja, Rojo

    fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.5, marker_colors=colors,
                                textinfo='label+percent', insidetextorientation='radial')])
    
    periodo_analizado = f"Periodo: {fecha_min.strftime('%d/%m/%Y')} - {fecha_max.strftime('%d/%m/%Y')}"
    fig.update_layout(
        title_text=f'Global distribution of operating time<br><sup>Total sample: {total_minutos} | {periodo_analizado}</sup>',
        title_x=0.5,
        annotations=[dict(text='Total', x=0.5, y=0.5, font_size=20, showarrow=False)],
        legend_title_text="Combined States",
        height=500
    )
    return fig.to_html(full_html=False, include_plotlyjs='cdn')

def analizar_performance_flags(df_total):
    """Analiza y grafica la performance de flags individuales."""
    if df_total.empty:
        return "<p class='warning-message'>Datos insuficientes para el análisis de performance de flags.</p>"
    
    flags_criticas = {
        'OPTIBAT_ON': {'desc': 'Active closed-loop', 'color': '#3498db'},
        'OPTIBAT_READY': {'desc': 'System ready', 'color': '#2ecc71'},
        'Macrostates_Flag_Copy': {'desc': 'Flag Macroestados OK', 'color': '#9b59b6'},
        'Support_Flag_Copy': {'desc': 'Flag Soporte OK', 'color': '#e67e22'},
        'Resultexistance_Flag_Copy': {'desc': 'Flag Existencia Resultado OK', 'color': '#f1c40f'},
        'OPTIBAT_WATCHDOG': {'desc': 'Watchdog Activo', 'color': '#1abc9c'},
        'Communication_ECS': {'desc': 'Comunicación ECS OK', 'color': '#e74c3c'} # Asumiendo 1 = OK
    }
    
    available_flags = [flag for flag in flags_criticas if flag in df_total.columns]
    if not available_flags:
        return "<p class='warning-message'>No se encontraron columnas de flags críticas para analizar.</p>"

    total_minutos = len(df_total)
    porcentajes = []
    nombres_flags = []
    colores_flags = []

    for flag_col in available_flags:
        if flag_col in df_total.columns:
            # Asumimos que 1 es el estado deseado/activo. Podría necesitarse lógica más compleja si 0 es deseado para alguna.
            tiempo_activo = df_total[flag_col].sum() # Suma ya que son 0s y 1s
            porcentaje_activo = (tiempo_activo / total_minutos * 100) if total_minutos > 0 else 0
            porcentajes.append(porcentaje_activo)
            nombres_flags.append(flags_criticas[flag_col]['desc'])
            colores_flags.append(flags_criticas[flag_col]['color'])

    if not porcentajes:
        return "<p class='warning-message'>No se pudieron calcular porcentajes para las flags.</p>"

    fig = go.Figure(data=[go.Bar(
        x=nombres_flags,
        y=porcentajes,
        text=[f'{p:.1f}%' for p in porcentajes],
        textposition='auto',
        marker_color=colores_flags
    )])
    
    fig.update_layout(
        title_text='Percentage of Uptime for Critical System Flags',
        title_x=0.5,
        xaxis_title="System flags",
        yaxis_title="Percentage of Time in Active State/OK (%)",
        yaxis_range=[0,100],
        height=500
    )
    return fig.to_html(full_html=False, include_plotlyjs='cdn')


# --- Función procesar_archivos (adaptada para el nuevo layout de dashboard) ---
def procesar_archivos(archivos, progress_window_ref, progressbar_widget, status_label_widget, general_info_label_widget, total_files):
    html_elements = [f"<h1>📊 Monthly OPTIBAT Dashboard Analysis (v{VERSION_REPORTE}) 📈</h1>"]
    todos_los_datos_para_global = []; resumen_metricas_para_global = []; todas_las_caidas_para_global = []
    total_steps = total_files + 4 # Archivos + consolidación + graficos_globales + HTML_final
    fecha_min_overall = None; fecha_max_overall = None

    for i, archivo_path in enumerate(archivos):
        current_step = i + 1; progress_percentage = (current_step / total_steps) * 100
        nombre_base_archivo = os.path.basename(archivo_path)
        
        if progress_window_ref.winfo_exists():
            general_info_label_widget.config(text="Procesando Archivos...")
            status_label_widget.config(text=f"Archivo {i+1}/{total_files}: {nombre_base_archivo}")
            progressbar_widget.config(value=progress_percentage)
            progress_window_ref.update_idletasks()
        else: # Ventana de progreso cerrada, abortar
            print("[INFO] Ventana de progreso cerrada por el usuario. Abortando análisis.")
            return # Salir de la función si la ventana de progreso ya no existe.

        try:
            df = cargar_archivo(archivo_path) # Usa el cargar_archivo de pandas
            if df.empty:
                html_elements.append(f"<p class='warning-message'>Archivo {nombre_base_archivo} está vacío o no pudo ser cargado.</p>")
                continue
            if 'Date' not in df.columns:
                print(f"Advertencia: Columna 'Date' no encontrada en {nombre_base_archivo}")
                html_elements.append(f"<p class='warning-message'>Columna 'Date' no encontrada en {nombre_base_archivo}. Saltando archivo.</p>")
                continue
            df['Date'] = pd.to_datetime(df['Date'], errors='coerce'); df.dropna(subset=['Date'], inplace=True)
            if df.empty:
                html_elements.append(f"<p class='warning-message'>Archivo {nombre_base_archivo} no contiene datos válidos después de procesar fechas.</p>")
                continue
            
            current_min_date = df['Date'].min(); current_max_date = df['Date'].max()
            if fecha_min_overall is None or current_min_date < fecha_min_overall: fecha_min_overall = current_min_date
            if fecha_max_overall is None or current_max_date > fecha_max_overall: fecha_max_overall = current_max_date
            
            columnas_numericas_necesarias = ['OPTIBAT_ON', 'OPTIBAT_READY', 'Macrostates_Flag_Copy', 'Support_Flag_Copy', 'Resultexistance_Flag_Copy', 'OPTIBAT_WATCHDOG', 'Communication_ECS']
            for col in columnas_numericas_necesarias:
                if col in df.columns: df[col] = pd.to_numeric(df[col], errors='coerce')
            
            df_calc = df.copy()
            for col_fill in ['OPTIBAT_ON', 'OPTIBAT_READY']:
                if col_fill in df_calc.columns: df_calc[col_fill] = df_calc[col_fill].fillna(0).astype(int)
            
            minutos_totales_archivo = len(df_calc)
            minutos_lazo_cerrado = df_calc['OPTIBAT_ON'].sum() if 'OPTIBAT_ON' in df_calc else 0
            minutos_lazo_abierto = minutos_totales_archivo - minutos_lazo_cerrado
            OPTIBAT_READY_1 = df_calc['OPTIBAT_READY'].sum() if 'OPTIBAT_READY' in df_calc else 0
            OPTIBAT_READY_0 = minutos_totales_archivo - OPTIBAT_READY_1

            flag_caidas_archivo = identificar_OPTIBAT_READY_caidas(df.copy())
            for caida in flag_caidas_archivo: caida['Archivo'] = nombre_base_archivo
            todas_las_caidas_para_global.extend(flag_caidas_archivo)

            cols_to_keep = ['Date', 'OPTIBAT_ON', 'OPTIBAT_READY'] + [col for col in ['Macrostates_Flag_Copy', 'Support_Flag_Copy', 'Resultexistance_Flag_Copy', 'OPTIBAT_WATCHDOG', 'Communication_ECS'] if col in df.columns]
            
            # Asegurarse que las columnas básicas existen antes de añadirlas para el agregado global
            df_to_append = df.copy()
            for esencial_col in ['Date', 'OPTIBAT_ON', 'OPTIBAT_READY']:
                if esencial_col not in df_to_append.columns:
                     df_to_append[esencial_col] = 0 # o np.nan, dependiendo del manejo posterior
            
            todos_los_datos_para_global.append(df_to_append[cols_to_keep])

            resumen_metricas_para_global.append({"Archivo": nombre_base_archivo, "FechaInicio": df['Date'].min(), "OPTIBAT_ON_1": minutos_lazo_cerrado, "OPTIBAT_ON_0": minutos_lazo_abierto, "OPTIBAT_READY_1": OPTIBAT_READY_1, "OPTIBAT_READY_0": OPTIBAT_READY_0})
        except Exception as e_file_processing:
            print(f"Error procesando el archivo {nombre_base_archivo} para datos globales: {e_file_processing}"); print(traceback.format_exc())
            html_elements.append(f"<div class='file-section error-message'><strong>Error al procesar el archivo {nombre_base_archivo} para agregación de datos:</strong><br/><pre>{traceback.format_exc()}</pre></div>")
            continue

    # --- SECCIONES GLOBALES DEL DASHBOARD ---
    if progress_window_ref.winfo_exists():
        general_info_label_widget.config(text="Generando Gráficos Globales...")
        progressbar_widget.config(value=((total_files + 1) / total_steps) * 100)
        progress_window_ref.update_idletasks()
    else: return

    df_total_agregado = pd.DataFrame()
    if todos_los_datos_para_global:
        try:
            df_total_agregado = pd.concat(todos_los_datos_para_global).sort_values(by='Date').reset_index(drop=True)
        except Exception as e_concat:
            print(f"Error al concatenar DataFrames globales: {e_concat}")
            html_elements.append(f"<p class='error-message'>Error al consolidar datos globales: {e_concat}</p>")


    if not df_total_agregado.empty and fecha_min_overall and fecha_max_overall:
        html_elements.append("<div class='dashboard-section summary-section'><h2>Global Summary of OPTIBAT Operation</h2>" + generar_grafico_rosquilla_global(df_total_agregado.copy(), fecha_min_overall, fecha_max_overall) + "</div>")
        html_elements.append("<div class='dashboard-section summary-section'><h2>System Flags Performance Analysis</h2>" + analizar_performance_flags(df_total_agregado.copy()) + "</div>")
    else: html_elements.append("<div class='dashboard-section summary-section'><p class='warning-message'>No hay datos suficientes para el resumen global de operación o análisis de flags.</p></div>")

    if todas_las_caidas_para_global:
        html_elements.append("<div class='dashboard-section summary-section'><h2>📉 Global Analysis of OPTIBAT_READY Down</h2>")
        graficos_causas_dict = generar_grafico_causas(todas_las_caidas_para_global)
        html_elements.append(graficos_causas_dict.get('pastel_anillo', '<p class="error-message">Gráfico pastel/anillo de causas no disponible.</p>'))
        html_elements.append(graficos_causas_dict.get('barras_horizontales', '<p class="error-message">Gráfico de barras de causas no disponible.</p>'))
        html_elements.append(graficos_causas_dict.get('boxplot_duracion', '<p class="error-message">Boxplot de duración de causas no disponible.</p>'))
        html_elements.append(generar_grafico_duracion_caidas(todas_las_caidas_para_global))
        html_elements.append(analizar_patrones_temporales(todas_las_caidas_para_global))
        html_elements.append("</div>")

    df_resumen_global_final = pd.DataFrame(resumen_metricas_para_global)
    if not df_resumen_global_final.empty:
        html_elements.append("<div class='dashboard-section summary-section'><h2>📊 Comparative Performance Summary per File</h2>" + generar_graficos_resumen(df_resumen_global_final.copy()) + "</div>")
        html_elements.append("<div class='dashboard-section summary-section'><h2>📈 Evolution of the System Over Time</h2>" + analizar_evolucion_sistema(todos_los_datos_para_global, resumen_metricas_para_global) + "</div>")

    if not df_total_agregado.empty:
        html_elements.append("<div class='dashboard-section summary-section'><h2>🌍 Global Interactive Visualization and Numerical Summary</h2>")
        if 'OPTIBAT_ON' in df_total_agregado.columns and 'Date' in df_total_agregado.columns:
            html_elements.append(graficar_interactivo_con_duracion(df_total_agregado.copy(), "Global - All combined Files"))
        else:
             html_elements.append("<p class='warning-message'>Columnas 'Date' u 'OPTIBAT_ON' faltantes para gráfico interactivo global.</p>")

        total_min_glob = len(df_total_agregado)
        # Asegurar que las columnas existen antes de usarlas
        ready1_on1_val, ready1_on0_val, ready0_val = 0, 0, 0
        if 'OPTIBAT_READY' in df_total_agregado.columns and 'OPTIBAT_ON' in df_total_agregado.columns:
            ready1_on1_val = len(df_total_agregado[(df_total_agregado['OPTIBAT_READY']==1) & (df_total_agregado['OPTIBAT_ON']==1)])
            ready1_on0_val = len(df_total_agregado[(df_total_agregado['OPTIBAT_READY']==1) & (df_total_agregado['OPTIBAT_ON']==0)])
            ready0_val = len(df_total_agregado[df_total_agregado['OPTIBAT_READY']==0])
        elif 'OPTIBAT_READY' in df_total_agregado.columns: # Solo OPTIBAT_READY disponible
             ready0_val = len(df_total_agregado[df_total_agregado['OPTIBAT_READY']==0])
             # No se puede calcular R1O1 ni R1O0
        
        if total_min_glob > 0:
            tabla_res_glob_data = [["Total analyzed samples", total_min_glob]]
            if 'OPTIBAT_READY' in df_total_agregado.columns and 'OPTIBAT_ON' in df_total_agregado.columns:
                 tabla_res_glob_data.extend([
                    ["✔️ OK (Ready=1 & ON=1)", f"{ready1_on1_val} sample ({ready1_on1_val/total_min_glob*100:.2f}%)"],
                    ["❌ NO OK (Ready=1 & ON=0)", f"{ready1_on0_val} sample ({ready1_on0_val/total_min_glob*100:.2f}%)"],
                    ["⛔ Unavailable (Ready=0)", f"{ready0_val} sample ({ready0_val/total_min_glob*100:.2f}%)"]
                 ])
            elif 'OPTIBAT_READY' in df_total_agregado.columns:
                 tabla_res_glob_data.append(["⛔ No disponible (Ready=0)", f"{ready0_val} sample ({ready0_val/total_min_glob*100:.2f}%)"])
            else:
                 tabla_res_glob_data.append(["Datos de Flags no disponibles para resumen", "-"])
            html_elements.append("<h3>Global Leverage Summary</h3>" + tabulate(tabla_res_glob_data, headers=["Category", "Value"], tablefmt="html"))
        html_elements.append("</div>")

    if progress_window_ref.winfo_exists():
        general_info_label_widget.config(text="Finalizando Análisis...")
        status_label_widget.config(text="Generando reporte HTML final...")
        progressbar_widget.config(value=total_steps / total_steps * 99) # Casi 100%
        progress_window_ref.update_idletasks()
    else: return


    final_html_content = f"""
    <html><head><meta charset="UTF-8"><title>Dashboard de Análisis OPTIBAT v{VERSION_REPORTE}</title>
    <style>
        body {{ font-family: 'Segoe UI', Arial, sans-serif; margin: 0; padding: 0; background-color: #eef1f5; color: #333; line-height: 1.65; }}
        .container {{ max-width: 1600px; margin: 20px auto; padding: 20px; background-color: #fff; box-shadow: 0 4px 20px rgba(0,0,0,0.12); border-radius: 10px; }}
        h1 {{ color: #2c3e50; text-align: center; border-bottom: 4px solid #3498db; padding-bottom: 18px; margin-top: 10px; margin-bottom: 35px; font-size: 2.4em; font-weight: 600;}}
        h2 {{ color: #3498db; margin-top: 35px; border-bottom: 2px solid #e0e0e0; padding-bottom: 12px; font-size: 1.8em; font-weight: 500;}}
        h3 {{ color: #2980b9; margin-top: 25px; font-size: 1.4em; border-left: 5px solid #3498db; padding-left: 12px; font-weight: 500;}}
        table {{ border-collapse: collapse; width: auto; margin: 20px 0; box-shadow: 0 1px 3px rgba(0,0,0,0.1); min-width: 50%; }} /* Ajustado width y min-width */
        table.GeneratedTable {{ width: 100%; }} /* Para tablas de tabulate */
        th, td {{ border: 1px solid #ccc; text-align: left; padding: 10px 12px; font-size: 0.9em; vertical-align: top; }}
        th {{ background-color: #3498db; color: white; font-weight: 600; white-space: nowrap; }}
        tr:nth-child(even) {{ background-color: #f8f9fa; }}
        tr:hover {{ background-color: #e9ecef; }}
        .dashboard-section {{ border-bottom: 2px dashed #bdc3c7; padding-bottom: 25px; margin-bottom:30px; }}
        .summary-section {{ padding: 15px; background-color: #f7f9fc; border-radius: 5px; border: 1px solid #e1e5ea;}}
        .plotly-graph-div, img {{ margin: 20px auto !important; border: 1px solid #ddd; box-shadow: 0 2px 8px rgba(0,0,0,0.05); display: block; border-radius: 4px; }}
        .warning-message {{ color: #856404; background-color: #fff3cd; border-color: #ffeeba; padding: 12px; margin:15px 0; border-left: 5px solid #ffc107; border-radius: 4px;}}
        .error-message {{ color: #721c24; background-color: #f8d7da; border-color: #f5c6cb; padding: 12px; margin:15px 0; border-left: 5px solid #dc3545; border-radius: 5px;}}
        .error-message strong {{font-size: 1.15em;}}
        .error-message pre {{ white-space: pre-wrap; font-size: 0.88em; max-height: 280px; overflow-y: auto; background: #f1f1f1; padding:8px 10px; border: 1px solid #ddd; border-radius:4px; margin-top:8px;}}
        details > summary {{ cursor: pointer; font-weight: bold; color: #555; margin-bottom: 5px; }}
        ul li {{ margin-bottom: 0.5em; }}
        footer {{ text-align:center; margin-top:40px; padding-top:20px; border-top:1px solid #ccc; font-size:0.85em; color:#777; }}
    </style></head><body><div class="container">
    {''.join(html_elements)}
    <footer>Reporte generado el {datetime.now().strftime('%d/%m/%Y %H:%M:%S')} por Analizador OPTIBAT v{VERSION_REPORTE}. Desarrollado por Juan Cruz E.</footer>
    </div></body></html>
    """
    report_filename = f"reporte_optibat_v{VERSION_REPORTE}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"; # Nombre único
    report_path = os.path.join(os.getcwd(), report_filename); abs_report_path = os.path.realpath(report_path); file_url = f"file://{abs_report_path}"
    try:
        with open(report_path, "w", encoding="utf-8") as f: f.write(final_html_content)
        print(f"\n[INFO] Reporte HTML generado en: {report_path}"); opened_successfully = False; print(f"[INFO] Intentando abrir URL: {file_url}")
        
        # Intentos de abrir el archivo
        try:
            if webbrowser.open(file_url, new=2, autoraise=True): opened_successfully = True
        except webbrowser.Error: pass # Ignorar errores de webbrowser por ahora
        
        if not opened_successfully:
            try:
                current_system = platform.system()
                if current_system == "Windows": os.startfile(abs_report_path); opened_successfully = True
                elif current_system == "Darwin": subprocess.run(['open', file_url], check=False); opened_successfully = True
                else: subprocess.run(['xdg-open', file_url], check=False); opened_successfully = True
            except Exception: pass # Ignorar errores de comandos de sistema

        if progress_window_ref.winfo_exists():
            final_gui_message = f"Reporte generado en:\n{abs_report_path}"
            if not opened_successfully:
                final_gui_message += "\n(No se pudo abrir automáticamente)"
            
            general_info_label_widget.config(text="¡Análisis Completado!") # MODIFICACIÓN
            status_label_widget.config(text=final_gui_message, foreground="darkgreen" if opened_successfully else "darkorange")
            progressbar_widget.config(value=100)
            progress_window_ref.update_idletasks()

    except IOError as e_io:
        print(f"Error de IO al escribir el reporte: {e_io}")
        if progress_window_ref.winfo_exists():
            general_info_label_widget.config(text="Error al Guardar Reporte", foreground="red")
            status_label_widget.config(text=f"No se pudo escribir el reporte HTML:\n{e_io}", foreground="red")
        raise IOError(f"No se pudo escribir el reporte HTML: {e_io}") from e_io
    except Exception as e_final:
        print(f"Error inesperado durante la generación/apertura final del reporte: {e_final}");
        if progress_window_ref.winfo_exists():
            general_info_label_widget.config(text="Error Final Inesperado", foreground="red")
            status_label_widget.config(text=f"Error: {e_final}", foreground="red")
        raise

# --- Función Main (GUI Principal Mejorada) ---
def main():
    main_root_window = tk.Tk(); VERSION_REPORTE_GUI = VERSION_REPORTE; main_root_window.title(f"Analizador de Datos OPTIBAT v{VERSION_REPORTE_GUI}"); main_root_window.resizable(False, False)
    style = ttk.Style(main_root_window); themes = style.theme_names()
    if "vista" in themes: style.theme_use("vista");
    elif "clam" in themes: style.theme_use("clam")
    elif "aqua" in themes: style.theme_use("aqua")
    else: style.theme_use("default")
    COLOR_BG = "#ECEFF1"; COLOR_FRAME_BG = "#FFFFFF"; COLOR_BANNER_BG = "#37474F"; COLOR_BANNER_FG = "#FFFFFF"; COLOR_BUTTON_BG = "#0277BD"; COLOR_BUTTON_FG = "#FFFFFF"; COLOR_BUTTON_ACTIVE_BG = "#01579B"; COLOR_TEXT = "#37474F"; COLOR_SECONDARY_TEXT = "#546E7A"
    main_root_window.configure(bg=COLOR_BG)
    container = ttk.Frame(main_root_window, padding="25 25 25 25", style="Container.TFrame"); container.pack(expand=True, fill='both'); style.configure("Container.TFrame", background=COLOR_FRAME_BG)
    banner_frame = tk.Frame(container, bg=COLOR_BANNER_BG, padx=15, pady=20); banner_frame.pack(fill='x', pady=(0, 25))
    tk.Label(banner_frame, text="Analizador Avanzado de Datos OPTIBAT", font=("Segoe UI Variable", 22, "bold"), fg=COLOR_BANNER_FG, bg=COLOR_BANNER_BG).pack()
    info_text = ("Esta herramienta analiza datos de operación de sistemas OPTIBAT.\n\n1. Presione 'Cargar y Analizar Archivos'.\n2. Seleccione los archivos de datos (.txt, .osf).\n3. Esta ventana se cerrará y se mostrará el progreso.\n4. El reporte HTML final se intentará abrir automáticamente.")
    info_label = tk.Label(container, text=info_text, font=("Segoe UI Variable", 11), wraplength=600, justify="left", fg=COLOR_TEXT, bg=COLOR_FRAME_BG, anchor="w", padx=10); info_label.pack(fill='x', pady=(0, 20))
    ttk.Separator(container, orient='horizontal').pack(fill='x', pady=15)
    
    # Se llama a la función cargar_archivos (la nueva función GUI)
    btn_cargar = tk.Button(container, text="Cargar y Analizar Archivos", command=lambda: cargar_archivos(main_root_window), font=("Segoe UI Variable", 14, "bold"), bg=COLOR_BUTTON_BG, fg=COLOR_BUTTON_FG, activebackground=COLOR_BUTTON_ACTIVE_BG, activeforeground=COLOR_BUTTON_FG, relief=tk.FLAT, padx=25, pady=12, borderwidth=0, highlightthickness=0, cursor="hand2"); btn_cargar.pack(pady=(15, 10))
    
    def on_enter_main(e): e.widget['background'] = COLOR_BUTTON_ACTIVE_BG
    def on_leave_main(e): e.widget['background'] = COLOR_BUTTON_BG
    btn_cargar.bind("<Enter>", on_enter_main); btn_cargar.bind("<Leave>", on_leave_main)
    buttons_row_frame = tk.Frame(container, bg=COLOR_FRAME_BG); buttons_row_frame.pack(pady=(15, 20))
    style.configure("Secondary.TButton", font=("Segoe UI Variable", 10), padding="10 5")
    def show_detailed_help():
        help_text = (f"AYUDA - ANALIZADOR OPTIBAT v{VERSION_REPORTE_GUI}\n\nFuncionamiento:\n- Al presionar 'Cargar y Analizar Archivos', podrá seleccionar múltiples archivos de datos.\n- La ventana actual se ocultará y aparecerá una nueva con el progreso del análisis.\n\nReporte Final:\n- Se generará un archivo HTML (ej: 'reporte_optibat_v{VERSION_REPORTE_GUI}_FECHAHORA.html') en la misma carpeta del script.\n- El script intentará abrirlo automáticamente en su navegador.\n- Contendrá un dashboard con los análisis configurados.\n\nErrores:\n- Si un archivo falla, se indicará en el reporte (si aplica) y se continuará con los demás.\n- Errores detallados también se muestran en la consola.")
        messagebox.showinfo("Ayuda Detallada", help_text, parent=main_root_window)
    btn_ayuda = ttk.Button(buttons_row_frame, text="Ayuda", command=show_detailed_help, style="Secondary.TButton", width=12); btn_ayuda.pack(side=tk.LEFT, padx=15)
    btn_salir = ttk.Button(buttons_row_frame, text="Salir", command=main_root_window.destroy, style="Secondary.TButton", width=12); btn_salir.pack(side=tk.LEFT, padx=15)
    footer_label = ttk.Label(container, text=f"v{VERSION_REPORTE_GUI} - Desarrollado por Juan Cruz E.", font=("Segoe UI Variable", 8), foreground=COLOR_SECONDARY_TEXT, background=COLOR_FRAME_BG); footer_label.pack(side=tk.BOTTOM, pady=(15,0))
    
    main_root_window.update_idletasks()
    width = 680; height = 550
    x = (main_root_window.winfo_screenwidth() // 2) - (width // 2)
    y = (main_root_window.winfo_screenheight() // 2) - (height // 2)
    main_root_window.geometry(f'{width}x{height}+{x}+{y}')

    def on_closing_main_window():
        if messagebox.askokcancel("Salir", "¿Desea salir de la aplicación?", parent=main_root_window):
            main_root_window.destroy()
    main_root_window.protocol("WM_DELETE_WINDOW", on_closing_main_window)

    main_root_window.mainloop()

if __name__ == "__main__":
    main()

[INFO] DPI Awareness establecido (shcore).


C:\Users\AntonioMéndez\AppData\Local\Temp\ipykernel_4524\672577873.py:293: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.


C:\Users\AntonioMéndez\AppData\Local\Temp\ipykernel_4524\672577873.py:331: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.


C:\Users\AntonioMéndez\AppData\Local\Temp\ipykernel_4524\672577873.py:384: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.


C:\Users\AntonioMéndez\AppData\Local\Temp\ipykernel_4524\672577873.py:400: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and se


[INFO] Reporte HTML generado en: c:\Users\AntonioMéndez\Desktop\MTTO\Maintenance report template\reporte_optibat_v3.2.1_20250703_122143.html
[INFO] Intentando abrir URL: file://C:\Users\AntonioMéndez\Desktop\MTTO\Maintenance report template\reporte_optibat_v3.2.1_20250703_122143.html


In [10]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
from tabulate import tabulate
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import seaborn as sns
import webbrowser
import io
import base64
import traceback
import platform
import subprocess

# --- DEFINICIÓN GLOBAL DE LA VERSIÓN DEL REPORTE ---
VERSION_REPORTE = "3.2.1"

# --- Intento de hacer la aplicación DPI-aware en Windows ---
if platform.system() == "Windows":
    try:
        from ctypes import windll
        windll.shcore.SetProcessDpiAwareness(1); print("[INFO] DPI Awareness establecido (shcore).")
    except Exception:
        try: windll.user32.SetProcessDPIAware(); print("[INFO] DPI Awareness establecido (user32).")
        except Exception as e_dpi: print(f"[ADVERTENCIA] No se pudo establecer el DPI awareness: {e_dpi}")

# --- FUNCIÓN AUXILIAR PARA GRÁFICOS MATPLOTLIB A HTML ---
def convert_plt_to_html(fig=None):
    if fig is None: fig = plt.gcf()
    buf = io.BytesIO(); fig.savefig(buf, format='png', bbox_inches='tight', dpi=150)
    buf.seek(0); img_str = base64.b64encode(buf.read()).decode('utf-8')
    plt.close(fig)
    return f'<img src="data:image/png;base64,{img_str}" alt="Matplotlib Graph" style="max-width:95%; height:auto; border:1px solid #d1d8e0; margin:20px auto; display:block; box-shadow: 0 3px 8px rgba(0,0,0,0.07); border-radius: 4px;"/>'

# --- FUNCIONES DE CARGA Y ANÁLISIS ---
def detectar_separador(primera_linea):
    posibles_separadores = ['\t', ';', ',', '|']; max_count = 0; mejor_separador = '\t'
    for sep in posibles_separadores:
        count = primera_linea.count(sep)
        if count > max_count: max_count = count; mejor_separador = sep
    return mejor_separador

def cargar_archivo(archivo): # Este es el cargar_archivo para leer el CSV
    try:
        num_lines = 0
        try:
            with open(archivo, 'r', encoding='utf-8', errors='ignore') as f_check:
                for _ in f_check: num_lines += 1
        except Exception: pass
        with open(archivo, 'r', encoding='utf-8', errors='ignore') as f:
            primeras_lineas = [next(f) for _ in range(min(20, num_lines if num_lines > 0 else 20))]
        separador = detectar_separador(primeras_lineas[0] if primeras_lineas else '\t'); header_row = 0
        header_row_found = False 
        for i, linea in enumerate(primeras_lineas):
            if any(key in linea.lower() for key in ['date', 'OPTIBAT_READY', 'optibat', 'timestamp', 'datetime']):
                header_row = i; header_row_found = True; break
        if not header_row_found and primeras_lineas :
                 header_row = 0

        csv_params = {'sep': separador, 'encoding': 'utf-8', 'engine': 'python', 'header': header_row, 'on_bad_lines': 'skip'}
        df = pd.read_csv(archivo, **csv_params)

        for col_name_variant in ['datetime', 'Timestamp', 'timestamp', 'Fecha', 'Date time', 'Time', 'time (UTC)']:
            if col_name_variant in df.columns:
                if 'Date' not in df.columns: 
                    df.rename(columns={col_name_variant: 'Date'}, inplace=True)
                    print(f"[INFO] Columna '{col_name_variant}' renombrada a 'Date' en archivo {os.path.basename(archivo)}")
                    break
                elif col_name_variant != 'Date': 
                    print(f"[INFO] Columna 'Date' ya existe. Columna '{col_name_variant}' será ignorada o puedes decidir eliminarla si es redundante.")
        return df
    except Exception as e: print(f"Error crítico al cargar el archivo {archivo}: {e}"); return pd.DataFrame()

def verificar_variacion_ininterrumpida(df, columna, max_repeticiones=4):
    repeticiones = 1; valor_anterior = None; problemas = []
    if columna not in df.columns or df[columna].empty: return problemas
    try: col_data = pd.to_numeric(df[columna], errors='coerce')
    except Exception: return problemas
    for idx, valor in enumerate(col_data):
        if pd.isna(valor) and pd.isna(valor_anterior): repeticiones +=1
        elif valor == valor_anterior and not pd.isna(valor): repeticiones += 1
        else: repeticiones = 1
        if repeticiones > max_repeticiones:
            start_idx = max(0, idx - (repeticiones - 1))
            if 'Date' in df.columns and start_idx < len(df) and idx < len(df):
                tiempo_inicio = pd.to_datetime(df['Date'].iloc[start_idx], errors='coerce')
                tiempo_fin = pd.to_datetime(df['Date'].iloc[idx], errors='coerce')
                if pd.notna(tiempo_inicio) and pd.notna(tiempo_fin):
                    problemas.append({"valor": valor_anterior, "repeticiones": repeticiones, "inicio": tiempo_inicio, "fin": tiempo_fin})
        valor_anterior = valor
    return problemas

def detectar_valores_nulos(df, inicio_OPTIBAT_READY_0=None):
    if inicio_OPTIBAT_READY_0 is None or 'Date' not in df.columns: return []
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    inicio_OPTIBAT_READY_0_dt = pd.to_datetime(inicio_OPTIBAT_READY_0, errors='coerce')
    if pd.isna(inicio_OPTIBAT_READY_0_dt): return []
    resultados = []; mascara = df['Date'] >= inicio_OPTIBAT_READY_0_dt; df_despues_caida = df.loc[mascara].copy()
    for columna in df.columns:
        if columna == 'Date': continue
        nulos_antes = df.loc[~mascara, columna].isnull().sum(); nulos_despues = df_despues_caida[columna].isnull().sum()
        total_antes = (~mascara).sum(); total_despues = mascara.sum()
        porc_nulos_antes = (nulos_antes / total_antes * 100) if total_antes > 0 else 0
        porc_nulos_despues = (nulos_despues / total_despues * 100) if total_despues > 0 else 0
        if porc_nulos_despues > porc_nulos_antes + 10 and nulos_despues > 5: resultados.append({"columna": columna, "nulos_antes": nulos_antes, "nulos_despues": nulos_despues, "porcentaje_antes": porc_nulos_antes, "porcentaje_despues": porc_nulos_despues, "incremento": porc_nulos_despues - porc_nulos_antes})
    return resultados

def identificar_OPTIBAT_READY_caidas(df, umbral_estancamiento=4):
    caidas = []
    estado_actual = None
    inicio_caida = None
    df_proc = df.copy() 
    cols_to_numeric = ['OPTIBAT_READY','FM2_COMMS_HeartBeat', 'Support_Flag_Copy', 'Macrostates_Flag_Copy',
                       'Resultexistance_Flag_Copy', 'Communication_ECS', 'OPTIBAT_WATCHDOG']
    for col in cols_to_numeric:
        if col in df_proc.columns:
            df_proc[col] = pd.to_numeric(df_proc[col], errors='coerce')

    if 'FM2_COMMS_HeartBeat' in df_proc.columns:
        df_proc['FM2_COMMS_HeartBeat_Estable'] = df_proc['FM2_COMMS_HeartBeat'].diff().fillna(0).abs() > 0

    if 'OPTIBAT_READY' not in df_proc.columns or 'Date' not in df_proc.columns:
        return caidas

    df_proc['Date'] = pd.to_datetime(df_proc['Date'], errors='coerce')
    df_proc.dropna(subset=['Date'], inplace=True)

    for idx, row in df_proc.iterrows():
        if pd.isna(row['OPTIBAT_READY']): continue

        causa_primaria = "Unknown"

        if row['OPTIBAT_READY'] == 0 and estado_actual != 0:
            estado_actual = 0
            inicio_caida = row['Date']
            if idx > 0:
                ventana = 5; inicio_ventana = max(0, idx - ventana)
                df_ventana = df_proc.iloc[inicio_ventana:idx+1]
                cambios = []
                if 'Support_Flag_Copy' in df_ventana.columns and (df_ventana['Support_Flag_Copy'].diff().fillna(1) != 0).any():
                    for i_v, cambio_v in enumerate(df_ventana['Support_Flag_Copy'].diff().fillna(1).ne(0)):
                        if cambio_v and df_ventana['Support_Flag_Copy'].iloc[i_v] == 0:
                            cambios.append(("Support_Flag_Copy", inicio_ventana + i_v))
                # (AÑADE TU LÓGICA COMPLETA AQUÍ PARA TODAS LAS FLAGS Y CONDICIONES)
                if cambios:
                    cambios.sort(key=lambda x: x[1])
                    causa_primaria = cambios[0][0]
                else:
                    causa_primaria = "Caída sin cambio de flag obvio en ventana"
            else:
                causa_primaria = "Inicio de archivo con OPTIBAT_READY=0"

        elif row['OPTIBAT_READY'] == 1 and estado_actual == 0:
            if inicio_caida is None: 
                estado_actual = 1
                continue

            fin_caida = row['Date']
            mask = (df_proc['Date'] >= inicio_caida) & (df_proc['Date'] <= fin_caida)
            subset = df_proc[mask]
            causas_periodo_actual = [] 
            if 'Support_Flag_Copy' in subset.columns and (subset['Support_Flag_Copy'] == 0).any(): causas_periodo_actual.append("Support_Flag_Copy=0")
            # (Tu lógica detallada para determinar causas_periodo)

            if not causas_periodo_actual: causas_periodo_actual = ["Causa desconocida en periodo"]
            detalles_nulos_info = detectar_valores_nulos(df, inicio_caida)

            caidas.append({
                "Inicio": pd.to_datetime(inicio_caida), "Fin": pd.to_datetime(fin_caida),
                "Duración (min)": len(subset), "Causas": " | ".join(causas_periodo_actual),
                "Causa Primaria": causa_primaria, "Detalles Nulos": detalles_nulos_info
            })
            estado_actual = 1; inicio_caida = None

    if estado_actual == 0 and inicio_caida is not None: 
        fin_caida = df_proc['Date'].iloc[-1]
        subset_final = df_proc[df_proc['Date'] >= inicio_caida] 
        causas_periodo_final = []
        if 'Support_Flag_Copy' in subset_final.columns and (subset_final['Support_Flag_Copy'] == 0).any(): causas_periodo_final.append("Support_Flag_Copy=0")
        if not causas_periodo_final: causas_periodo_final = ["Caída hasta fin de archivo"]
        # (Lógica similar para el último periodo de caída)
        caidas.append({
            "Inicio": pd.to_datetime(inicio_caida), "Fin": pd.to_datetime(fin_caida),
            "Duración (min)": len(subset_final),
            "Causas": " | ".join(causas_periodo_final), "Causa Primaria": causa_primaria,
            "Detalles Nulos": detectar_valores_nulos(df, inicio_caida)
        })
    return caidas

def analizar_valores_nulos(df):
    return {"tabla": "", "graficos": ""}

# --- FUNCIONES DE GRAFICADO ---
def graficar_interactivo_con_duracion(df, titulo):
    df_plot = df[['Date', 'OPTIBAT_ON']].copy(); df_plot.dropna(inplace=True); df_plot.reset_index(drop=True, inplace=True)
    if df_plot.empty: return "<p class='warning-message'>No hay datos para el gráfico interactivo OPTIBAT_ON.</p>"
    df_plot['Date'] = pd.to_datetime(df_plot['Date']); df_plot = df_plot.sort_values(by="Date").reset_index(drop=True); df_plot['OPTIBAT_ON'] = pd.to_numeric(df_plot['OPTIBAT_ON'], errors='coerce').fillna(0)
    segmentos = []; cambios = df_plot['OPTIBAT_ON'].diff().fillna(1).ne(0); indices_cambio = df_plot.index[cambios].tolist()
    if not df_plot.empty and (not indices_cambio or indices_cambio[-1] != df_plot.index[-1]): indices_cambio.append(df_plot.index[-1] + 1)
    start_idx = 0
    for end_idx in indices_cambio:
        if start_idx >= end_idx or start_idx >= len(df_plot): break
        current_end_idx = min(end_idx, len(df_plot))
        segment_df = df_plot.iloc[start_idx : current_end_idx];
        if segment_df.empty: continue
        start_date = segment_df['Date'].iloc[0]
        if len(segment_df) > 1: end_date_seg = segment_df['Date'].iloc[-1]; duracion_minutos = round((end_date_seg - start_date).total_seconds() / 60) +1
        else: end_date_seg = start_date; duracion_minutos = 1
        value = segment_df['OPTIBAT_ON'].iloc[0]; midpoint_date = start_date + (end_date_seg - start_date) / 2
        segmentos.append((start_date, end_date_seg, value, duracion_minutos, midpoint_date)); start_idx = end_idx
    fig = go.Figure(); fig.add_trace(go.Scatter(x=df_plot['Date'], y=df_plot['OPTIBAT_ON'], mode='lines', line_shape='hv', name='OPTIBAT_ON', line=dict(color='royalblue', width=2.5)))
    for s_start, s_end, s_val, s_mins, s_mid in segmentos:
        if s_mins >= 10: fig.add_annotation(x=s_mid, y=s_val, text=f"{s_mins} min", showarrow=False, yshift=10 if s_val == 1 else -20, font=dict(size=10, color='black'), bgcolor="rgba(255,255,0,0.6)", borderpad=2)
    tiempo_on = (df_plot['OPTIBAT_ON'] == 1).sum(); tiempo_off = (df_plot['OPTIBAT_ON'] == 0).sum(); total_tiempo = len(df_plot)
    porcentaje_on = (tiempo_on / total_tiempo * 100) if total_tiempo > 0 else 0
    estadisticas = (f"<b>Total:</b> {total_tiempo} min<br><b>Closed Loop:</b> {tiempo_on} min ({porcentaje_on:.1f}%)<br><b>Open Loop:</b> {tiempo_off} min ({100-porcentaje_on:.1f}%)")
    fig.add_annotation(x=0.01, y=0.99, xref="paper", yref="paper", text=estadisticas, showarrow=False, font=dict(size=10), align="left", bgcolor="rgba(240,240,240,0.85)", bordercolor="#b0b0b0", borderwidth=1, borderpad=4)
    fig.update_layout(title=dict(text=f"Loop state (OPTIBAT_ON) - {titulo}", x=0.5, font=dict(size=18)), yaxis_tickvals=[0, 1], yaxis_ticktext=['Open Loop (0)', 'Closed Loop (1)'], height=550, margin=dict(l=50, r=30, t=70, b=50), legend_title_text='Estado')
    fig.update_xaxes(title_text='Date and time', rangeslider_visible=True); fig.update_yaxes(title_text='Loop state')
    return fig.to_html(full_html=False, include_plotlyjs='cdn')

def generar_graficos_resumen(df_graficos):
    if df_graficos.empty: return "<p class='warning-message'>No hay datos para los gráficos de resumen por archivo.</p>"
    html_outputs = []
    df_graficos_copy = df_graficos.copy()
    df_graficos_copy.sort_values(by='FechaInicio', inplace=True)
    df_graficos_copy['Archivo'] = df_graficos_copy['Archivo'].apply(lambda x: os.path.basename(str(x)).split('.')[0])

    plt.figure(figsize=(12, 6.5))
    p1 = plt.bar(df_graficos_copy["Archivo"], df_graficos_copy["OPTIBAT_ON_1"], label="Lazo cerrado (1)", color="#1f77b4", width=0.7)
    p2 = plt.bar(df_graficos_copy["Archivo"], df_graficos_copy["OPTIBAT_ON_0"], bottom=df_graficos_copy["OPTIBAT_ON_1"], label="Lazo abierto (0)", color="#ff7f0e", width=0.7)
    plt.title("Distribución de Tiempo OPTIBAT_ON por Archivo", fontsize=15, weight='bold', pad=12); plt.xlabel("Archivo", fontsize=11); plt.ylabel("Minutos", fontsize=11); plt.xticks(rotation=45, ha='right', fontsize=8.5); plt.yticks(fontsize=9); plt.legend(fontsize=9, loc='upper right'); plt.grid(axis='y', linestyle='--', alpha=0.7)
    for i, (rect1, rect2) in enumerate(zip(p1, p2)):
        h1=rect1.get_height(); h2=rect2.get_height(); total = h1+h2
        if h1 > 50 : plt.text(rect1.get_x() + rect1.get_width()/2., h1/2, f'{int(h1)}', ha='center', va='center', color='white', fontsize=4, weight='bold')
        if h2 > 50 : plt.text(rect2.get_x() + rect2.get_width()/2., h1 + h2/2, f'{int(h2)}', ha='center', va='center', color='white', fontsize=4, weight='bold')
        if total > 0: plt.text(rect1.get_x() + rect1.get_width()/2., total + plt.ylim()[1]*0.01, f'{int(total)}', ha='center', va='bottom', fontsize=4)
    avg_cerrado = df_graficos_copy["OPTIBAT_ON_1"].mean(); avg_abierto = df_graficos_copy["OPTIBAT_ON_0"].mean(); total_promedio = avg_cerrado + avg_abierto
    if total_promedio > 0: stats_text = (f"Promedio por archivo:\nLazo Cerrado: {avg_cerrado:.1f} min ({avg_cerrado/total_promedio*100:.1f}%)\nLazo Abierto: {avg_abierto:.1f} min ({avg_abierto/total_promedio*100:.1f}%)"); plt.text(0.01, 0.01, stats_text, transform=plt.gca().transAxes, fontsize=8.5, va='bottom', bbox=dict(boxstyle='round,pad=0.3', fc='aliceblue', alpha=0.8))
    plt.tight_layout(pad=1.5); html_outputs.append(convert_plt_to_html(plt.gcf()))

    plt.figure(figsize=(11, 6.5)); p1f = plt.bar(df_graficos_copy["Archivo"], df_graficos_copy["OPTIBAT_READY_1"], label="Ready=1", color="#2ca02c", width=0.7); p2f = plt.bar(df_graficos_copy["Archivo"], df_graficos_copy["OPTIBAT_READY_0"], bottom=df_graficos_copy["OPTIBAT_READY_1"], label="Ready=0", color="#d62728", width=0.7)
    plt.title("Distribución de Tiempo OPTIBAT_READY por Archivo", fontsize=15, weight='bold', pad=12); plt.xlabel("Archivo", fontsize=11); plt.ylabel("Minutos", fontsize=11); plt.xticks(rotation=45, ha='right', fontsize=8.5); plt.yticks(fontsize=9); plt.legend(fontsize=9, loc='upper right'); plt.grid(axis='y', linestyle='--', alpha=0.7)
    for i, (rect1, rect2) in enumerate(zip(p1f, p2f)):
        h1=rect1.get_height(); h2=rect2.get_height(); total = h1+h2
        if h1 > 50 : plt.text(rect1.get_x() + rect1.get_width()/2., h1/2, f'{int(h1)}', ha='center', va='center', color='white', fontsize=4, weight='bold')
        if h2 > 50 : plt.text(rect2.get_x() + rect2.get_width()/2., h1 + h2/2, f'{int(h2)}', ha='center', va='center', color='white', fontsize=4, weight='bold')
        if total > 0: plt.text(rect1.get_x() + rect1.get_width()/2., total + plt.ylim()[1]*0.01, f'{int(total)}', ha='center', va='bottom', fontsize=4)
    avg_ready1 = df_graficos_copy["OPTIBAT_READY_1"].mean(); avg_ready0 = df_graficos_copy["OPTIBAT_READY_0"].mean(); total_promedio_ready = avg_ready1 + avg_ready0
    if total_promedio_ready > 0: stats_text_ready = (f"Promedio por archivo:\nReady=1: {avg_ready1:.1f} min ({avg_ready1/total_promedio_ready*100:.1f}%)\nReady=0: {avg_ready0:.1f} min ({avg_ready0/total_promedio_ready*100:.1f}%)"); plt.text(0.01, 0.01, stats_text_ready, transform=plt.gca().transAxes, fontsize=8.5, va='bottom', bbox=dict(boxstyle='round,pad=0.3', fc='honeydew', alpha=0.8))
    plt.tight_layout(pad=1.5); html_outputs.append(convert_plt_to_html(plt.gcf()))

    plt.figure(figsize=(10, 6)); porcentajes = []
    for _, row in df_graficos_copy.iterrows(): total_optibat = row['OPTIBAT_ON_0'] + row['OPTIBAT_ON_1']; porcentajes.append((row['OPTIBAT_ON_1'] / total_optibat * 100) if total_optibat > 0 else 0)
    barras_porc = plt.bar(df_graficos_copy["Archivo"], porcentajes, color="#8c564b", width=0.6); promedio_porc = np.mean(porcentajes) if porcentajes else 0
    plt.axhline(y=promedio_porc, color='red', linestyle='--', linewidth=1.5, label=f'Average: {promedio_porc:.1f}%'); meta_lazo_cerrado = 90; plt.axhline(y=meta_lazo_cerrado, color='green', linestyle=':', linewidth=1.5, label=f'Meta: {meta_lazo_cerrado}%')
    plt.title("Percentage of time in closed loop per file", fontsize=15, weight='bold', pad=12); plt.xlabel("File", fontsize=11); plt.ylabel("Percentage (%)", fontsize=11); plt.xticks(rotation=45, ha='right', fontsize=8.5); plt.yticks(np.arange(0, 101, 10), fontsize=9); plt.ylim(0, 105)
    for i, bar in enumerate(barras_porc): height = bar.get_height(); plt.text(bar.get_x() + bar.get_width()/2., height + 1, f'{height:.1f}%', ha='center', va='bottom', fontsize=4.2, weight='bold');
    plt.legend(fontsize=9); plt.grid(axis='y', linestyle=':', alpha=0.7); plt.tight_layout(pad=1.5); html_outputs.append(convert_plt_to_html(plt.gcf()))
    return "\n".join(html_outputs)

def generar_grafico_causas(todas_las_caidas):
    graficos_html = {'barras_horizontales': '', 'pastel_anillo': '', 'boxplot_duracion': ''}
    if not todas_las_caidas:
        no_data_msg = "<p class='warning-message'>No hay datos de caídas para generar gráficos de causas.</p>"
        return {key: no_data_msg for key in graficos_html}
    
    df_caidas = pd.DataFrame(todas_las_caidas)
    if df_caidas.empty or 'Causa Primaria' not in df_caidas.columns or 'Duración (min)' not in df_caidas.columns:
        return {key: "<p class='warning-message'>Datos de caídas incompletos para gráficos.</p>" for key in graficos_html}

    try:
        plt.figure(figsize=(10, 8))
        causas_counts = df_caidas['Causa Primaria'].value_counts()
        sns.barplot(y=causas_counts.index, x=causas_counts.values, palette="viridis", orient='h')
        plt.title('Frecuencia de Causas Primarias de Caída de OPTIBAT_READY', fontsize=14, pad=10)
        plt.xlabel('Number of Occurrences', fontsize=11)
        plt.ylabel('Causa Primaria', fontsize=11)
        plt.xticks(fontsize=9); plt.yticks(fontsize=9); plt.tight_layout()
        graficos_html['barras_horizontales'] = convert_plt_to_html()
    except Exception as e: graficos_html['barras_horizontales'] = f"<p class='error-message'>Error generando gráfico de barras: {e}</p>"

    try:
        plt.figure(figsize=(8, 8))
        top_n = 7
        if len(causas_counts) > top_n:
            otras_sum = causas_counts[top_n:].sum()
            causas_plot = causas_counts.head(top_n).copy()
            if otras_sum > 0: causas_plot['Otras (< min freq.)'] = otras_sum
        else: causas_plot = causas_counts.copy()
        plt.pie(causas_plot, labels=causas_plot.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel"), wedgeprops=dict(width=0.4, edgecolor='w'))
        plt.title('Distribución Porcentual de Causas Primarias', fontsize=14, pad=15); plt.axis('equal')
        graficos_html['pastel_anillo'] = convert_plt_to_html()
    except Exception as e: graficos_html['pastel_anillo'] = f"<p class='error-message'>Error generando gráfico de pastel: {e}</p>"
        
    try:
        plt.figure(figsize=(12, 7))
        common_causas = df_caidas['Causa Primaria'].value_counts().nlargest(10).index
        df_caidas_common = df_caidas[df_caidas['Causa Primaria'].isin(common_causas)]
        sns.boxplot(data=df_caidas_common, x='Duración (min)', y='Causa Primaria', palette="Set2", orient='h')
        plt.title('Distribution of down Duration by Primary Cause (Top 10)', fontsize=14, pad=10)
        plt.xlabel('Down duration (minutes)', fontsize=11); plt.ylabel('Primary Cause', fontsize=11)
        plt.xscale('log'); plt.xticks(fontsize=9); plt.yticks(fontsize=9); plt.grid(axis='x', linestyle='--', alpha=0.6); plt.tight_layout()
        graficos_html['boxplot_duracion'] = convert_plt_to_html()
    except Exception as e: graficos_html['boxplot_duracion'] = f"<p class='error-message'>Error generando boxplot de duración: {e}</p>"
    return graficos_html

def generar_grafico_duracion_caidas(todas_las_caidas):
    if not todas_las_caidas: return "<p class='warning-message'>No hay datos para gráficos de duración de caídas.</p>"
    df_caidas = pd.DataFrame(todas_las_caidas)
    if df_caidas.empty or 'Duración (min)' not in df_caidas.columns: return "<p class='warning-message'>Datos de duración de caídas incompletos.</p>"
    html_output = []
    try:
        plt.figure(figsize=(10, 6))
        sns.histplot(df_caidas['Duración (min)'], kde=True, bins=30, color='skyblue')
        plt.title('Distribución de la Duración de las Caídas de OPTIBAT_READY', fontsize=14); plt.xlabel('Duración (minutos)', fontsize=11); plt.ylabel('Frecuencia', fontsize=11)
        plt.yscale('log'); plt.grid(axis='y', linestyle='--', alpha=0.7); plt.tight_layout()
        html_output.append(convert_plt_to_html())
    except Exception as e: html_output.append(f"<p class='error-message'>Error generando histograma de duración: {e}</p>")
    return "\n".join(html_output)

def analizar_patrones_temporales(todas_las_caidas):
    if not todas_las_caidas: return "<p class='warning-message'>No hay datos para analizar patrones temporales de caídas.</p>"
    df_caidas = pd.DataFrame(todas_las_caidas)
    if df_caidas.empty or 'Inicio' not in df_caidas.columns: return "<p class='warning-message'>Datos de inicio de caídas incompletos para patrones temporales.</p>"
    df_caidas['Inicio'] = pd.to_datetime(df_caidas['Inicio']); html_output = []
    try:
        plt.figure(figsize=(10, 6)); df_caidas['Hora_Inicio'] = df_caidas['Inicio'].dt.hour
        sns.countplot(data=df_caidas, x='Hora_Inicio', palette='Spectral')
        plt.title('Number of OPTIBAT_READY down per Time of Day', fontsize=14); plt.xlabel('Time of day (0-23)', fontsize=11); plt.ylabel('Number of downs', fontsize=11)
        plt.xticks(np.arange(0, 24, 1)); plt.grid(axis='y', linestyle='--', alpha=0.7); plt.tight_layout()
        html_output.append(convert_plt_to_html())
    except Exception as e: html_output.append(f"<p class='error-message'>Error generando gráfico de caídas por hora: {e}</p>")
    try:
        plt.figure(figsize=(10, 6)); df_caidas['Dia_Semana_Inicio'] = df_caidas['Inicio'].dt.day_name()
        days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        sns.countplot(data=df_caidas, x='Dia_Semana_Inicio', order=days_order, palette='Paired')
        plt.title('Number of OPTIBAT_READY down per day of week', fontsize=14); plt.xlabel('Day of the week', fontsize=11); plt.ylabel('Number of downs', fontsize=11)
        plt.xticks(rotation=45, ha='right'); plt.grid(axis='y', linestyle='--', alpha=0.7); plt.tight_layout()
        html_output.append(convert_plt_to_html())
    except Exception as e: html_output.append(f"<p class='error-message'>Error generando gráfico de caídas por día de la semana: {e}</p>")
    return "\n".join(html_output)

def analizar_evolucion_sistema(todos_los_datos_df_list, resumen_graficos_list):
    if not resumen_graficos_list: return "<p class='warning-message'>No hay datos para analizar la evolución del sistema.</p>"
    df_resumen = pd.DataFrame(resumen_graficos_list)
    if df_resumen.empty or 'FechaInicio' not in df_resumen.columns: return "<p class='warning-message'>Datos de resumen incompletos para evolución.</p>"
    df_resumen['FechaInicio'] = pd.to_datetime(df_resumen['FechaInicio']); df_resumen.sort_values('FechaInicio', inplace=True); html_output = []
    try:
        if 'OPTIBAT_ON_1' in df_resumen.columns and 'OPTIBAT_ON_0' in df_resumen.columns:
            df_resumen['Porcentaje_Lazo_Cerrado'] = (df_resumen['OPTIBAT_ON_1'] / (df_resumen['OPTIBAT_ON_1'] + df_resumen['OPTIBAT_ON_0'])).fillna(0) * 100
            plt.figure(figsize=(12, 6)); plt.plot(df_resumen['FechaInicio'], df_resumen['Porcentaje_Lazo_Cerrado'], marker='o', linestyle='-', color='teal')
            plt.title('Evolución del Porcentaje de Tiempo en Lazo Cerrado (OPTIBAT_ON=1)', fontsize=14); plt.xlabel('Fecha', fontsize=11); plt.ylabel('Porcentaje de Lazo Cerrado (%)', fontsize=11)
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d')); plt.xticks(rotation=45, ha='right'); plt.ylim(0, 105); plt.grid(True, linestyle='--', alpha=0.7); plt.tight_layout()
            html_output.append(convert_plt_to_html())
    except Exception as e: html_output.append(f"<p class='error-message'>Error generando gráfico de evolución de lazo cerrado: {e}</p>")
    try:
        if 'OPTIBAT_READY_1' in df_resumen.columns and 'OPTIBAT_READY_0' in df_resumen.columns:
            df_resumen['Porcentaje_OPTIBAT_READY_1'] = (df_resumen['OPTIBAT_READY_1'] / (df_resumen['OPTIBAT_READY_1'] + df_resumen['OPTIBAT_READY_0'])).fillna(0) * 100
            plt.figure(figsize=(12, 6)); plt.plot(df_resumen['FechaInicio'], df_resumen['Porcentaje_OPTIBAT_READY_1'], marker='s', linestyle='-', color='coral')
            plt.title('Evolución del Porcentaje de Tiempo con OPTIBAT_READY=1', fontsize=14); plt.xlabel('Fecha', fontsize=11); plt.ylabel('Porcentaje de OPTIBAT_READY=1 (%)', fontsize=11)
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d')); plt.xticks(rotation=45, ha='right'); plt.ylim(0, 105); plt.grid(True, linestyle='--', alpha=0.7); plt.tight_layout()
            html_output.append(convert_plt_to_html())
    except Exception as e: html_output.append(f"<p class='error-message'>Error generando gráfico de evolución de OPTIBAT_READY: {e}</p>")
    return "\n".join(html_output)

# --- FUNCIONES AUXILIARES PARA LA GUI DE PROGRESO ---
def close_progress_and_exit(progress_win, main_win):
    if progress_win.winfo_exists():
        progress_win.destroy()
    if main_win.winfo_exists():
        main_win.destroy()

# --- FUNCIÓN cargar_archivos (para la GUI) ---
def cargar_archivos(main_window_ref):
    archivos_seleccionados = filedialog.askopenfilenames(
        title="Seleccionar archivos de datos",
        filetypes=(("Archivos de Texto", "*.txt"), ("Archivos OSF", "*.osf"), ("Todos los archivos", "*.*")),
        parent=main_window_ref
    )
    if not archivos_seleccionados: return

    main_window_ref.withdraw()

    progress_window = tk.Toplevel(bg="#F0F0F0")
    progress_window.title("Procesando Archivos...")
    width, height = 450, 200
    x_prog = (progress_window.winfo_screenwidth() // 2) - (width // 2)
    y_prog = (progress_window.winfo_screenheight() // 2) - (height // 2)
    progress_window.geometry(f'{width}x{height}+{x_prog}+{y_prog}')
    progress_window.resizable(False, False)
    progress_window.grab_set()

    def on_closing_progress_window():
        if progress_window.winfo_exists(): # Solo si aún existe
             # Cancelar cualquier 'after' pendiente para evitar errores si se cierra manualmente
            try:
                # No hay una forma directa de cancelar todos los after jobs por ID si no lo guardamos
                # Simplemente cerraremos. Si 'procesar_archivos' está en curso, seguirá hasta el próximo check.
                pass
            except tk.TclError: # Puede ocurrir si la ventana ya está en proceso de destrucción
                pass

        if messagebox.askokcancel("Salir", "¿Interrumpir el análisis y salir de la aplicación?", parent=progress_window if progress_window.winfo_exists() else None):
            close_progress_and_exit(progress_window, main_window_ref)
    progress_window.protocol("WM_DELETE_WINDOW", on_closing_progress_window)

    prog_frame = ttk.Frame(progress_window, padding="20 20 20 20")
    prog_frame.pack(expand=True, fill='both')

    prog_general_info_label = ttk.Label(prog_frame, text="Preparando análisis...", font=("Segoe UI", 11))
    prog_general_info_label.pack(pady=(0,10), anchor="w")

    prog_status_label = ttk.Label(prog_frame, text="Archivos pendientes...", font=("Segoe UI", 10), foreground="gray", wraplength=width-40)
    prog_status_label.pack(pady=(0,15), anchor="w")
    
    progressbar = ttk.Progressbar(prog_frame, orient="horizontal", length=400, mode="determinate", value=0)
    progressbar.pack(pady=(0,20), fill='x')

    button_frame_prog = ttk.Frame(prog_frame) 

    # --- INICIO SECCIÓN MODIFICADA PARA CIERRE AUTOMÁTICO ---
    try:
        procesar_archivos(
            archivos_seleccionados,
            progress_window,
            progressbar,
            prog_status_label,
            prog_general_info_label,
            len(archivos_seleccionados)
        )
        
        if progress_window.winfo_exists():
            current_status_message = prog_status_label.cget("text")
            prog_status_label.config(text=f"{current_status_message}\n\nEsta ventana se cerrará automáticamente en 5 segundos.")
            progress_window.update_idletasks()
            delay_ms = 5000 
            # Guardar el ID del job para poder cancelarlo si es necesario
            # (aunque no se use explícitamente en on_closing_progress_window por simplicidad)
            after_id = progress_window.after(delay_ms, lambda: close_progress_and_exit(progress_window, main_window_ref))
            
    except Exception as e_proc:
        print(f"Error crítico capturado en cargar_archivos durante la llamada a procesar_archivos: {e_proc}")
        traceback.print_exc()
        if progress_window.winfo_exists():
            prog_general_info_label.config(text="Error Crítico Durante el Análisis", foreground="red")
            prog_status_label.config(text=f"Detalle del error: {str(e_proc)[:150]}...\nConsulte la consola para más detalles.", foreground="red")
            
            button_frame_prog.pack(pady=(10,0)) 
            btn_cerrar_error = ttk.Button(button_frame_prog, text="Cerrar para salir", command=lambda: close_progress_and_exit(progress_window, main_window_ref))
            btn_cerrar_error.pack()
        else: 
            if main_window_ref.winfo_exists() and not main_window_ref.winfo_viewable(): # Solo si estaba oculta
                messagebox.showerror("Error Crítico en Procesamiento", f"Ocurrió un error no manejado (ventana de progreso cerrada):\n{e_proc}")
            if main_window_ref.winfo_exists(): main_window_ref.destroy()
    # --- FIN SECCIÓN MODIFICADA ---

# --- FUNCIONES AUXILIARES PARA procesar_archivos ---
def generar_grafico_rosquilla_global(df_total, fecha_min, fecha_max):
    if df_total.empty or 'OPTIBAT_ON' not in df_total.columns or 'OPTIBAT_READY' not in df_total.columns:
        return "<p class='warning-message'>Datos insuficientes para el gráfico de rosquilla global.</p>"
    total_minutos = len(df_total)
    on_ready = len(df_total[(df_total['OPTIBAT_ON'] == 1) & (df_total['OPTIBAT_READY'] == 1)])
    on_not_ready = len(df_total[(df_total['OPTIBAT_ON'] == 1) & (df_total['OPTIBAT_READY'] == 0)])
    off_ready = len(df_total[(df_total['OPTIBAT_ON'] == 0) & (df_total['OPTIBAT_READY'] == 1)])
    off_not_ready = len(df_total[(df_total['OPTIBAT_ON'] == 0) & (df_total['OPTIBAT_READY'] == 0)])
    labels = ['ON & Ready', 'OFF & Ready', 'No Ready (ON u OFF)']
    values = [on_ready, off_ready, on_not_ready + off_not_ready]
    colors = ['#2ecc71', '#f39c12', '#e74c3c']
    fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.5, marker_colors=colors, textinfo='label+percent', insidetextorientation='radial')])
    periodo_analizado = f"Period: {fecha_min.strftime('%d/%m/%Y')} - {fecha_max.strftime('%d/%m/%Y')}"
    fig.update_layout(title_text=f'Global distribution of operating time<br><sup>Total sample: {total_minutos} | {periodo_analizado}</sup>', title_x=0.5, annotations=[dict(text='Total', x=0.5, y=0.5, font_size=20, showarrow=False)], legend_title_text="Combined States", height=500)
    return fig.to_html(full_html=False, include_plotlyjs='cdn')

def analizar_performance_flags(df_total):
    if df_total.empty: return "<p class='warning-message'>Datos insuficientes para el análisis de performance de flags.</p>"
    flags_criticas = {'OPTIBAT_ON': {'desc': 'Active closed-loop', 'color': '#3498db'}, 'OPTIBAT_READY': {'desc': 'System Ready', 'color': '#2ecc71'}, 'Macrostates_Flag_Copy': {'desc': 'Flag Macroestados OK', 'color': '#9b59b6'}, 'Support_Flag_Copy': {'desc': 'Flag Soporte OK', 'color': '#e67e22'}, 'Resultexistance_Flag_Copy': {'desc': 'Flag Existencia Resultado OK', 'color': '#f1c40f'}, 'OPTIBAT_WATCHDOG': {'desc': 'Watchdog Activo', 'color': '#1abc9c'}, 'Communication_ECS': {'desc': 'Comunicación ECS OK', 'color': '#e74c3c'}}
    available_flags = [flag for flag in flags_criticas if flag in df_total.columns]
    if not available_flags: return "<p class='warning-message'>No se encontraron columnas de flags críticas para analizar.</p>"
    total_minutos = len(df_total); porcentajes = []; nombres_flags = []; colores_flags = []
    for flag_col in available_flags:
        if flag_col in df_total.columns:
            tiempo_activo = df_total[flag_col].fillna(0).astype(int).sum()
            porcentaje_activo = (tiempo_activo / total_minutos * 100) if total_minutos > 0 else 0
            porcentajes.append(porcentaje_activo); nombres_flags.append(flags_criticas[flag_col]['desc']); colores_flags.append(flags_criticas[flag_col]['color'])
    if not porcentajes: return "<p class='warning-message'>No se pudieron calcular porcentajes para las flags.</p>"
    fig = go.Figure(data=[go.Bar(x=nombres_flags, y=porcentajes, text=[f'{p:.1f}%' for p in porcentajes], textposition='auto', marker_color=colores_flags)])
    fig.update_layout(title_text='Percentage of Uptime for Critical System Flags', title_x=0.5, xaxis_title="System flag", yaxis_title="Percentage of Time in Active State/OK (%)", yaxis_range=[0,100], height=500)
    return fig.to_html(full_html=False, include_plotlyjs='cdn')

# --- Función procesar_archivos ---
def procesar_archivos(archivos, progress_window_ref, progressbar_widget, status_label_widget, general_info_label_widget, total_files):
    html_elements = [f"<h1>📊 Dashboard de Análisis OPTIBAT (v{VERSION_REPORTE}) 📈</h1>"]
    todos_los_datos_para_global = []; resumen_metricas_para_global = []; todas_las_caidas_para_global = []
    total_steps = total_files + 4 
    fecha_min_overall = None; fecha_max_overall = None

    for i, archivo_path in enumerate(archivos):
        if not progress_window_ref.winfo_exists(): print("[INFO] Ventana de progreso cerrada. Abortando análisis."); return
        current_step = i + 1; progress_percentage = (current_step / total_steps) * 100
        nombre_base_archivo = os.path.basename(archivo_path)
        general_info_label_widget.config(text="Procesando Archivos..."); status_label_widget.config(text=f"Archivo {i+1}/{total_files}: {nombre_base_archivo}")
        progressbar_widget.config(value=progress_percentage); progress_window_ref.update_idletasks()
        try:
            df = cargar_archivo(archivo_path)
            if df.empty: html_elements.append(f"<p class='warning-message'>Archivo {nombre_base_archivo} está vacío o no pudo ser cargado.</p>"); continue
            if 'Date' not in df.columns: print(f"Advertencia: Columna 'Date' no encontrada en {nombre_base_archivo}"); html_elements.append(f"<p class='warning-message'>Columna 'Date' no encontrada en {nombre_base_archivo}. Saltando archivo.</p>"); continue
            df['Date'] = pd.to_datetime(df['Date'], errors='coerce'); df.dropna(subset=['Date'], inplace=True)
            if df.empty: html_elements.append(f"<p class='warning-message'>Archivo {nombre_base_archivo} no contiene datos válidos después de procesar fechas.</p>"); continue
            current_min_date = df['Date'].min(); current_max_date = df['Date'].max()
            if fecha_min_overall is None or current_min_date < fecha_min_overall: fecha_min_overall = current_min_date
            if fecha_max_overall is None or current_max_date > fecha_max_overall: fecha_max_overall = current_max_date
            columnas_numericas_necesarias = ['OPTIBAT_ON', 'OPTIBAT_READY', 'Macrostates_Flag_Copy', 'Support_Flag_Copy', 'Resultexistance_Flag_Copy', 'OPTIBAT_WATCHDOG', 'Communication_ECS']
            for col in columnas_numericas_necesarias:
                if col in df.columns: df[col] = pd.to_numeric(df[col], errors='coerce')
            df_calc = df.copy()
            for col_fill in ['OPTIBAT_ON', 'OPTIBAT_READY']:
                if col_fill in df_calc.columns: df_calc[col_fill] = df_calc[col_fill].fillna(0).astype(int)
            minutos_totales_archivo = len(df_calc); minutos_lazo_cerrado = df_calc['OPTIBAT_ON'].sum() if 'OPTIBAT_ON' in df_calc else 0; minutos_lazo_abierto = minutos_totales_archivo - minutos_lazo_cerrado; OPTIBAT_READY_1 = df_calc['OPTIBAT_READY'].sum() if 'OPTIBAT_READY' in df_calc else 0; OPTIBAT_READY_0 = minutos_totales_archivo - OPTIBAT_READY_1
            flag_caidas_archivo = identificar_OPTIBAT_READY_caidas(df.copy())
            for caida in flag_caidas_archivo: caida['Archivo'] = nombre_base_archivo
            todas_las_caidas_para_global.extend(flag_caidas_archivo)
            cols_to_keep = ['Date', 'OPTIBAT_ON', 'OPTIBAT_READY'] + [col for col in ['Macrostates_Flag_Copy', 'Support_Flag_Copy', 'Resultexistance_Flag_Copy', 'OPTIBAT_WATCHDOG', 'Communication_ECS'] if col in df.columns]
            df_to_append = df.copy()
            for esencial_col in ['Date', 'OPTIBAT_ON', 'OPTIBAT_READY']:
                if esencial_col not in df_to_append.columns: df_to_append[esencial_col] = np.nan # Usar NaN para consistencia numérica/fecha
            todos_los_datos_para_global.append(df_to_append[cols_to_keep])
            resumen_metricas_para_global.append({"Archivo": nombre_base_archivo, "FechaInicio": df['Date'].min(), "OPTIBAT_ON_1": minutos_lazo_cerrado, "OPTIBAT_ON_0": minutos_lazo_abierto, "OPTIBAT_READY_1": OPTIBAT_READY_1, "OPTIBAT_READY_0": OPTIBAT_READY_0})
        except Exception as e_file_processing: print(f"Error procesando el archivo {nombre_base_archivo} para datos globales: {e_file_processing}"); print(traceback.format_exc()); html_elements.append(f"<div class='file-section error-message'><strong>Error al procesar el archivo {nombre_base_archivo} para agregación de datos:</strong><br/><pre>{traceback.format_exc()}</pre></div>"); continue

    if not progress_window_ref.winfo_exists(): print("[INFO] Ventana de progreso cerrada. Abortando generación de gráficos."); return
    general_info_label_widget.config(text="Generando Gráficos Globales..."); progressbar_widget.config(value=((total_files + 1) / total_steps) * 100); progress_window_ref.update_idletasks()
    df_total_agregado = pd.DataFrame()
    if todos_los_datos_para_global:
        try: df_total_agregado = pd.concat(todos_los_datos_para_global).sort_values(by='Date').reset_index(drop=True)
        except Exception as e_concat: print(f"Error al concatenar DataFrames globales: {e_concat}"); html_elements.append(f"<p class='error-message'>Error al consolidar datos globales: {e_concat}</p>")
    if not df_total_agregado.empty and fecha_min_overall and fecha_max_overall:
        html_elements.append("<div class='dashboard-section summary-section'><h2>Resumen Global de Operación OPTIBAT</h2>" + generar_grafico_rosquilla_global(df_total_agregado.copy(), fecha_min_overall, fecha_max_overall) + "</div>")
        html_elements.append("<div class='dashboard-section summary-section'><h2>Análisis de Performance de Flags del Sistema</h2>" + analizar_performance_flags(df_total_agregado.copy()) + "</div>")
    else: html_elements.append("<div class='dashboard-section summary-section'><p class='warning-message'>No hay datos suficientes para el resumen global de operación o análisis de flags.</p></div>")
    if todas_las_caidas_para_global:
        html_elements.append("<div class='dashboard-section summary-section'><h2>📉 Global Analysis of OPTIBAT_READY down</h2>")
        graficos_causas_dict = generar_grafico_causas(todas_las_caidas_para_global)
        html_elements.append(graficos_causas_dict.get('pastel_anillo', '<p class="error-message">Gráfico pastel/anillo de causas no disponible.</p>'))
        html_elements.append(graficos_causas_dict.get('barras_horizontales', '<p class="error-message">Gráfico de barras de causas no disponible.</p>'))
        html_elements.append(graficos_causas_dict.get('boxplot_duracion', '<p class="error-message">Boxplot de duración de causas no disponible.</p>'))
        html_elements.append(generar_grafico_duracion_caidas(todas_las_caidas_para_global))
        html_elements.append(analizar_patrones_temporales(todas_las_caidas_para_global)); html_elements.append("</div>")
    df_resumen_global_final = pd.DataFrame(resumen_metricas_para_global)
    if not df_resumen_global_final.empty:
        html_elements.append("<div class='dashboard-section summary-section'><h2>📊 Comparative Performance Summary per File</h2>" + generar_graficos_resumen(df_resumen_global_final.copy()) + "</div>")
        html_elements.append("<div class='dashboard-section summary-section'><h2>📈 Evolution of the System Over Time</h2>" + analizar_evolucion_sistema(todos_los_datos_para_global, resumen_metricas_para_global) + "</div>")
    if not df_total_agregado.empty:
        html_elements.append("<div class='dashboard-section summary-section'><h2>🌍 Global Interactive Visualization and Numerical Summary</h2>")
        if 'OPTIBAT_ON' in df_total_agregado.columns and 'Date' in df_total_agregado.columns: html_elements.append(graficar_interactivo_con_duracion(df_total_agregado.copy(), "Global - All combined files"))
        else: html_elements.append("<p class='warning-message'>Columnas 'Date' u 'OPTIBAT_ON' faltantes para gráfico interactivo global.</p>")
        total_min_glob = len(df_total_agregado); ready1_on1_val, ready1_on0_val, ready0_val = 0, 0, 0
        if 'OPTIBAT_READY' in df_total_agregado.columns and 'OPTIBAT_ON' in df_total_agregado.columns:
            ready1_on1_val = len(df_total_agregado[(df_total_agregado['OPTIBAT_READY']==1) & (df_total_agregado['OPTIBAT_ON']==1)])
            ready1_on0_val = len(df_total_agregado[(df_total_agregado['OPTIBAT_READY']==1) & (df_total_agregado['OPTIBAT_ON']==0)])
            ready0_val = len(df_total_agregado[df_total_agregado['OPTIBAT_READY']==0])
        elif 'OPTIBAT_READY' in df_total_agregado.columns: ready0_val = len(df_total_agregado[df_total_agregado['OPTIBAT_READY']==0])
        if total_min_glob > 0:
            tabla_res_glob_data = [["Total registros analizados", total_min_glob]]
            if 'OPTIBAT_READY' in df_total_agregado.columns and 'OPTIBAT_ON' in df_total_agregado.columns: tabla_res_glob_data.extend([["✔️ Aprovechado (Ready=1 & ON=1)", f"{ready1_on1_val} min ({ready1_on1_val/total_min_glob*100:.2f}%)"], ["❌ Desaprovechado (Ready=1 & ON=0)", f"{ready1_on0_val} min ({ready1_on0_val/total_min_glob*100:.2f}%)"], ["⛔ No disponible (Ready=0)", f"{ready0_val} min ({ready0_val/total_min_glob*100:.2f}%)"]])
            elif 'OPTIBAT_READY' in df_total_agregado.columns: tabla_res_glob_data.append(["⛔ No disponible (Ready=0)", f"{ready0_val} min ({ready0_val/total_min_glob*100:.2f}%)"])
            else: tabla_res_glob_data.append(["Datos de Flags no disponibles para resumen", "-"])
            html_elements.append("<h3>Global Leverage Summary</h3>" + tabulate(tabla_res_glob_data, headers=["Category", "Value"], tablefmt="html")); html_elements.append("</div>")
    
    if not progress_window_ref.winfo_exists(): print("[INFO] Ventana de progreso cerrada. Abortando generación de reporte."); return
    general_info_label_widget.config(text="Finalizando Análisis..."); status_label_widget.config(text="Generando reporte HTML final...")
    progressbar_widget.config(value=total_steps / total_steps * 99); progress_window_ref.update_idletasks()
    final_html_content = f"""<html><head><meta charset="UTF-8"><title>Dashboard de Análisis OPTIBAT v{VERSION_REPORTE}</title><style>body{{font-family:'Segoe UI',Arial,sans-serif;margin:0;padding:0;background-color:#eef1f5;color:#333;line-height:1.65;}}.container{{max-width:1600px;margin:20px auto;padding:20px;background-color:#fff;box-shadow:0 4px 20px rgba(0,0,0,0.12);border-radius:10px;}}h1{{color:#2c3e50;text-align:center;border-bottom:4px solid #3498db;padding-bottom:18px;margin-top:10px;margin-bottom:35px;font-size:2.4em;font-weight:600;}}h2{{color:#3498db;margin-top:35px;border-bottom:2px solid #e0e0e0;padding-bottom:12px;font-size:1.8em;font-weight:500;}}h3{{color:#2980b9;margin-top:25px;font-size:1.4em;border-left:5px solid #3498db;padding-left:12px;font-weight:500;}}table{{border-collapse:collapse;width:auto;margin:20px 0;box-shadow:0 1px 3px rgba(0,0,0,0.1);min-width:50%;}}table.GeneratedTable{{width:100%;}}th,td{{border:1px solid #ccc;text-align:left;padding:10px 12px;font-size:0.9em;vertical-align:top;}}th{{background-color:#3498db;color:white;font-weight:600;white-space:nowrap;}}tr:nth-child(even){{background-color:#f8f9fa;}}tr:hover{{background-color:#e9ecef;}}.dashboard-section{{border-bottom:2px dashed #bdc3c7;padding-bottom:25px;margin-bottom:30px;}}.summary-section{{padding:15px;background-color:#f7f9fc;border-radius:5px;border:1px solid #e1e5ea;}}.plotly-graph-div,img{{margin:20px auto !important;border:1px solid #ddd;box-shadow:0 2px 8px rgba(0,0,0,0.05);display:block;border-radius:4px;}}.warning-message{{color:#856404;background-color:#fff3cd;border-color:#ffeeba;padding:12px;margin:15px 0;border-left:5px solid #ffc107;border-radius:4px;}}.error-message{{color:#721c24;background-color:#f8d7da;border-color:#f5c6cb;padding:12px;margin:15px 0;border-left:5px solid #dc3545;border-radius:5px;}}.error-message strong{{font-size:1.15em;}}.error-message pre{{white-space:pre-wrap;font-size:0.88em;max-height:280px;overflow-y:auto;background:#f1f1f1;padding:8px 10px;border:1px solid #ddd;border-radius:4px;margin-top:8px;}}details > summary{{cursor:pointer;font-weight:bold;color:#555;margin-bottom:5px;}}ul li{{margin-bottom:0.5em;}}footer{{text-align:center;margin-top:40px;padding-top:20px;border-top:1px solid #ccc;font-size:0.85em;color:#777;}}</style></head><body><div class="container">{''.join(html_elements)}<footer>Reporte generado el {datetime.now().strftime('%d/%m/%Y %H:%M:%S')} por Analizador OPTIBAT v{VERSION_REPORTE}. Desarrollado por Juan Cruz E.</footer></div></body></html>"""
    report_filename = f"reporte_optibat_v{VERSION_REPORTE}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"; report_path = os.path.join(os.getcwd(), report_filename); abs_report_path = os.path.realpath(report_path); file_url = f"file://{abs_report_path}"
    try:
        with open(report_path, "w", encoding="utf-8") as f: f.write(final_html_content)
        print(f"\n[INFO] Reporte HTML generado en: {report_path}"); opened_successfully = False; print(f"[INFO] Intentando abrir URL: {file_url}")
        try:
            if webbrowser.open(file_url, new=2, autoraise=True): opened_successfully = True
        except webbrowser.Error: pass
        if not opened_successfully:
            try:
                current_system = platform.system()
                if current_system == "Windows": os.startfile(abs_report_path); opened_successfully = True
                elif current_system == "Darwin": subprocess.run(['open', file_url], check=False); opened_successfully = True
                else: subprocess.run(['xdg-open', file_url], check=False); opened_successfully = True
            except Exception: pass
        if progress_window_ref.winfo_exists():
            final_gui_message = f"Reporte generado en:\n{abs_report_path}"
            if not opened_successfully: final_gui_message += "\n(No se pudo abrir automáticamente)"
            general_info_label_widget.config(text="¡Análisis Completado!")
            status_label_widget.config(text=final_gui_message, foreground="darkgreen" if opened_successfully else "darkorange")
            progressbar_widget.config(value=100); progress_window_ref.update_idletasks()
    except IOError as e_io: print(f"Error de IO al escribir el reporte: {e_io}"); general_info_label_widget.config(text="Error al Guardar Reporte", foreground="red"); status_label_widget.config(text=f"No se pudo escribir el reporte HTML:\n{e_io}", foreground="red"); raise IOError(f"No se pudo escribir el reporte HTML: {e_io}") from e_io
    except Exception as e_final: print(f"Error inesperado durante la generación/apertura final del reporte: {e_final}"); general_info_label_widget.config(text="Error Final Inesperado", foreground="red"); status_label_widget.config(text=f"Error: {e_final}", foreground="red"); raise

# --- Función Main (GUI Principal) ---
def main():
    main_root_window = tk.Tk(); VERSION_REPORTE_GUI = VERSION_REPORTE; main_root_window.title(f"Analizador de Datos OPTIBAT v{VERSION_REPORTE_GUI}"); main_root_window.resizable(False, False)
    style = ttk.Style(main_root_window); themes = style.theme_names()
    if "vista" in themes: style.theme_use("vista");
    elif "clam" in themes: style.theme_use("clam")
    elif "aqua" in themes: style.theme_use("aqua")
    else: style.theme_use("default")
    COLOR_BG = "#ECEFF1"; COLOR_FRAME_BG = "#FFFFFF"; COLOR_BANNER_BG = "#37474F"; COLOR_BANNER_FG = "#FFFFFF"; COLOR_BUTTON_BG = "#0277BD"; COLOR_BUTTON_FG = "#FFFFFF"; COLOR_BUTTON_ACTIVE_BG = "#01579B"; COLOR_TEXT = "#37474F"; COLOR_SECONDARY_TEXT = "#546E7A"
    main_root_window.configure(bg=COLOR_BG)
    container = ttk.Frame(main_root_window, padding="25 25 25 25", style="Container.TFrame"); container.pack(expand=True, fill='both'); style.configure("Container.TFrame", background=COLOR_FRAME_BG)
    banner_frame = tk.Frame(container, bg=COLOR_BANNER_BG, padx=15, pady=20); banner_frame.pack(fill='x', pady=(0, 25))
    tk.Label(banner_frame, text="Analizador Avanzado de Datos OPTIBAT", font=("Segoe UI Variable", 22, "bold"), fg=COLOR_BANNER_FG, bg=COLOR_BANNER_BG).pack()
    info_text = ("Esta herramienta analiza datos de operación de sistemas OPTIBAT.\n\n1. Presione 'Cargar y Analizar Archivos'.\n2. Seleccione los archivos de datos (.txt, .osf).\n3. Esta ventana se ocultará y se mostrará el progreso.\n4. El reporte HTML final se intentará abrir automáticamente.")
    info_label = tk.Label(container, text=info_text, font=("Segoe UI Variable", 11), wraplength=600, justify="left", fg=COLOR_TEXT, bg=COLOR_FRAME_BG, anchor="w", padx=10); info_label.pack(fill='x', pady=(0, 20))
    ttk.Separator(container, orient='horizontal').pack(fill='x', pady=15)
    btn_cargar = tk.Button(container, text="Cargar y Analizar Archivos", command=lambda: cargar_archivos(main_root_window), font=("Segoe UI Variable", 14, "bold"), bg=COLOR_BUTTON_BG, fg=COLOR_BUTTON_FG, activebackground=COLOR_BUTTON_ACTIVE_BG, activeforeground=COLOR_BUTTON_FG, relief=tk.FLAT, padx=25, pady=12, borderwidth=0, highlightthickness=0, cursor="hand2"); btn_cargar.pack(pady=(15, 10))
    def on_enter_main(e): e.widget['background'] = COLOR_BUTTON_ACTIVE_BG
    def on_leave_main(e): e.widget['background'] = COLOR_BUTTON_BG
    btn_cargar.bind("<Enter>", on_enter_main); btn_cargar.bind("<Leave>", on_leave_main)
    buttons_row_frame = tk.Frame(container, bg=COLOR_FRAME_BG); buttons_row_frame.pack(pady=(15, 20))
    style.configure("Secondary.TButton", font=("Segoe UI Variable", 10), padding="10 5")
    def show_detailed_help():
        help_text = (f"AYUDA - ANALIZADOR OPTIBAT v{VERSION_REPORTE_GUI}\n\nFuncionamiento:\n- Al presionar 'Cargar y Analizar Archivos', podrá seleccionar múltiples archivos de datos.\n- La ventana actual se ocultará y aparecerá una nueva con el progreso del análisis.\n\nReporte Final:\n- Se generará un archivo HTML (ej: 'reporte_optibat_v{VERSION_REPORTE_GUI}_FECHAHORA.html') en la misma carpeta del script.\n- El script intentará abrirlo automáticamente en su navegador.\n- Si el análisis es exitoso, la ventana de progreso se cerrará automáticamente tras unos segundos.\n\nErrores:\n- Si un archivo falla, se indicará en el reporte (si aplica) y se continuará con los demás.\n- Errores detallados también se muestran en la consola. Si ocurre un error mayor, la ventana de progreso requerirá cierre manual.")
        messagebox.showinfo("Ayuda Detallada", help_text, parent=main_root_window)
    btn_ayuda = ttk.Button(buttons_row_frame, text="Ayuda", command=show_detailed_help, style="Secondary.TButton", width=12); btn_ayuda.pack(side=tk.LEFT, padx=15)
    btn_salir = ttk.Button(buttons_row_frame, text="Salir", command=main_root_window.destroy, style="Secondary.TButton", width=12); btn_salir.pack(side=tk.LEFT, padx=15)
    footer_label = ttk.Label(container, text=f"v{VERSION_REPORTE_GUI} - Desarrollado por Juan Cruz E.", font=("Segoe UI Variable", 8), foreground=COLOR_SECONDARY_TEXT, background=COLOR_FRAME_BG); footer_label.pack(side=tk.BOTTOM, pady=(15,0))
    main_root_window.update_idletasks(); width = 680; height = 550
    x = (main_root_window.winfo_screenwidth() // 2) - (width // 2); y = (main_root_window.winfo_screenheight() // 2) - (height // 2)
    main_root_window.geometry(f'{width}x{height}+{x}+{y}')
    def on_closing_main_window():
        if messagebox.askokcancel("Salir", "¿Desea salir de la aplicación?", parent=main_root_window): main_root_window.destroy()
    main_root_window.protocol("WM_DELETE_WINDOW", on_closing_main_window)
    main_root_window.mainloop()

if __name__ == "__main__":
    main()

[INFO] DPI Awareness establecido (shcore).


C:\Users\AntonioMéndez\AppData\Local\Temp\ipykernel_4524\142399259.py:282: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.


C:\Users\AntonioMéndez\AppData\Local\Temp\ipykernel_4524\142399259.py:307: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.


C:\Users\AntonioMéndez\AppData\Local\Temp\ipykernel_4524\142399259.py:336: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.


C:\Users\AntonioMéndez\AppData\Local\Temp\ipykernel_4524\142399259.py:344: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and se


[INFO] Reporte HTML generado en: c:\Users\AntonioMéndez\Desktop\MTTO\Maintenance report template\reporte_optibat_v3.2.1_20250703_121038.html
[INFO] Intentando abrir URL: file://C:\Users\AntonioMéndez\Desktop\MTTO\Maintenance report template\reporte_optibat_v3.2.1_20250703_121038.html
